**01Heartbeat Sound LSTM Classification**

## Heartbeat sounds

Heart sounds are produced from a specific cardiac event such as closure of a valve or tensing of a chordae tendineae.

• S1 result from the closing of the mitral and tricuspid valves.

• S2 produced by the closure of the aortic and pulmonic valves.

In medicine we call the ‘lub’ sound "S1" and the ‘dub’ sound "S2".

You can learn short intro about heart sounds from this video:

https://www.youtube.com/watch?v=dBwr2GZCmQM

Why using Deep Learning in this ?

It is a subset of Neural Networks ( a computing systems which is insprired by biological neural networks ). And by stacking a lot of Neural Networks layers, we will get a Deep Learning model which might contains multiple billions parameters and is capable of handle very complicated problems that no classical algorithm can be effective.

Deep Learning is especially powerful with unstructrured data (Image, Text, Sound, Video, …).

## About data:
Is challenge published in 2012 to classify the heart sound to some categories from ‘AISTATS’ . Data has been gathered from two sources (A) and (B).

A) from the general public via the iStethoscope Pro.

B) from a clinic trial in hospitals using the digital stethoscope DigiScope.

Before we work on this nootebook we handle the data folders and conctante both sources(A&B) to be easy to deal with it.

Original Data here: http://www.peterjbentley.com/heartchallenge/

## Import main libraries

In [ ]:
import os
import glob
import fnmatch
import pandas as pd
import numpy as np
import librosa #To deal with Audio files
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
import math
import tensorflow as tf

中文注释：

```python
import os  # 导入os模块，用于与操作系统交互，如文件和目录操作
import glob  # 导入glob模块，用于文件模式匹配，可以找到符合特定模式的文件路径
import fnmatch  # 导入fnmatch模块，用于文件名模式匹配，常用于字符串比较
import pandas as pd  # 导入pandas库，并使用别名pd，这是一个数据分析和操作的强大工具
import numpy as np  # 导入numpy库，并使用别名np，这是一个进行科学计算的基础库
import librosa  # 导入librosa库，用于音频文件的处理，如音频信号分析
import librosa.display  # 导入librosa的显示模块，用于可视化音频信号
import matplotlib.pyplot as plt  # 导入matplotlib的pyplot模块，并使用别名plt，用于绘图
import IPython.display as ipd  # 导入IPython的display模块，并使用别名ipd，用于显示多媒体内容
import math  # 导入math模块，提供了一系列数学函数，如三角函数、指数、对数等
import tensorflow as tf  # 导入tensorflow库，并使用别名tf，这是一个用于机器学习的开源框架
```

这些注释解释了每行代码导入的模块及其用途，它们为Python程序提供了处理文件、数据分析、音频处理、绘图、数学计算和机器学习等功能。

In [ ]:
data_path = "../input/heartbeat-sound/Heartbeat_Sound/"
print(os.listdir(data_path))

中文注释：

```python
data_path = "../input/heartbeat-sound/Heartbeat_Sound/"  # 定义一个字符串变量data_path，它包含了一个文件路径。这个路径指向一个名为"Heartbeat_Sound"的目录，该目录位于上一级的"input/heartbeat-sound"目录中。
print(os.listdir(data_path))  # 使用os模块的listdir函数，它接受一个目录路径作为参数，返回该目录下所有文件和子目录的列表。然后将这个列表打印出来，通常用于查看目录内容。
```

这段代码的作用是列出指定目录下的所有文件和子目录的名称。这对于了解目录结构或进行进一步的文件操作非常有用。

In [ ]:
tarin_data      = data_path 
unlabel_data        = data_path  + "/unlabel/"

normal_data     = tarin_data + '/normal/'
murmur_data     = tarin_data + '/murmur/'
extrastole_data = tarin_data + '/extrastole/'
artifact_data   = tarin_data + '/artifact/'
extrahls_data   = tarin_data + "/extrahls/"

中文注释：

```python
tarin_data = data_path  # 将之前定义的data_path变量赋值给tarin_data变量，作为训练数据集的路径。
unlabel_data = data_path + "/unlabel/"  # 将data_path与子目录"/unlabel/"拼接，形成未标记数据集的路径。

normal_data = tarin_data + '/normal/'  # 将tarin_data与子目录'/normal/'拼接，形成正常心跳声音数据集的路径。
murmur_data = tarin_data + '/murmur/'  # 将tarin_data与子目录'/murmur/'拼接，形成心脏杂音数据集的路径。
extrastole_data = tarin_data + '/extrastole/'  # 将tarin_data与子目录'/extrastole/'拼接，形成额外心动过速数据集的路径。
artifact_data = tarin_data + '/artifact/'  # 将tarin_data与子目录'/artifact/'拼接，形成伪迹数据集的路径。
extrahls_data = tarin_data + "/extrahls/"  # 将tarin_data与子目录"/extrahls/"拼接，形成额外高保真声音数据集的路径。
```

这段代码通过拼接字符串来构建不同的数据集路径。这些路径指向存放心跳声音文件的不同分类目录，用于后续的数据加载和处理。

In [ ]:
print("Normal files:", len(os.listdir(normal_data))) #length of normal training sounds
print("Murmur files:",len(os.listdir(murmur_data))) #length of murmur training sounds 
print("Extrastole files", len(os.listdir(extrastole_data))) #length of extrastole training sounds 
print("Artifact files:",len(os.listdir(artifact_data))) #length of artifact training sounds 
print("Extrahls files:",len(os.listdir(extrahls_data))) #length of extrahls training sounds 

print('TOTAL TRAIN SOUNDS:', len(os.listdir(normal_data)) 
                              + len(os.listdir(murmur_data))
                              + len(os.listdir(extrastole_data))
                              + len(os.listdir(artifact_data))
                              + len(os.listdir(extrahls_data)))

中文注释：

```python
print("Normal files:", len(os.listdir(normal_data)))  # 打印出正常心跳声音数据集的文件数量。使用os.listdir(normal_data)获取该目录下所有文件和目录的列表，然后计算列表长度。
print("Murmur files:", len(os.listdir(murmur_data)))  # 打印出心脏杂音数据集的文件数量。同样，使用os.listdir(murmur_data)获取列表并计算长度。
print("Extrastole files", len(os.listdir(extrastole_data)))  # 打印出额外心动过速数据集的文件数量。使用os.listdir(extrastole_data)获取列表并计算长度。
print("Artifact files:", len(os.listdir(artifact_data)))  # 打印出伪迹数据集的文件数量。使用os.listdir(artifact_data)获取列表并计算长度。
print("Extrahls files:", len(os.listdir(extrahls_data)))  # 打印出额外高保真声音数据集的文件数量。使用os.listdir(extrahls_data)获取列表并计算长度。

print('TOTAL TRAIN SOUNDS:', len(os.listdir(normal_data))  # 打印出所有训练声音文件的总数。
                              + len(os.listdir(murmur_data))  # 将各个数据集目录的文件数量相加。
                              + len(os.listdir(extrastole_data))  # 包括正常心跳、心脏杂音、额外心动过速、伪迹和额外高保真声音数据集。
                              + len(os.listdir(artifact_data))  # 计算总和，得到训练集的总文件数量。
                              + len(os.listdir(extrahls_data)))  # 完成总文件数量的计算并打印结果。
```

这段代码的目的是统计并打印出指定目录下各类心跳声音数据集的文件数量，以及所有训练声音文件的总数。这对于了解数据集的规模和准备数据处理非常有帮助。

In [ ]:
print("Test sounds: ", len(os.listdir(unlabel_data)))

中文注释：

```python
print("Test sounds: ", len(os.listdir(unlabel_data)))  # 打印出未标记测试数据集的文件数量。首先使用os.listdir(unlabel_data)获取unlabel_data路径下所有文件和目录的列表，然后计算该列表的长度，并将结果打印出来。列表的长度即代表了该目录下的文件和子目录的总数。
```

这段代码的作用是统计并打印出未标记测试数据集（unlabel_data）中的文件数量。这对于了解测试数据集的大小和进行后续的数据处理或分析工作很有帮助。

## EDA and Visualization

In [ ]:
x = np.array([len(os.listdir(normal_data)),
              len(os.listdir(murmur_data)),
              len(os.listdir(extrastole_data)),
              len(os.listdir(artifact_data)),
              len(os.listdir(extrahls_data))])
labels = ['normal', 'murmur', 'extrastole', 'artifact', 'extrahls']
plt.pie(x, labels = labels, autopct = '%.0f%%', radius= 1.5, textprops={'fontsize': 16})
plt.show()

中文注释：

```python
x = np.array([len(os.listdir(normal_data)),  # 使用numpy库创建一个数组x，数组中的元素是各个心跳声音数据子目录下的文件数量。
              len(os.listdir(murmur_data)),
              len(os.listdir(extrastole_data)),
              len(os.listdir(artifact_data)),
              len(os.listdir(extrahls_data))])  # 每个子目录的文件数量通过os.listdir()函数获取，并计算其长度。

labels = ['normal', 'murmur', 'extrastole', 'artifact', 'extrahls']  # 定义一个列表labels，包含了每个数组元素对应的类别标签。

plt.pie(x, labels = labels, autopct = '%.0f%%', radius= 1.5, textprops={'fontsize': 16})  # 使用matplotlib的pyplot模块绘制一个饼图。参数说明：
  - x: 饼图的数据来源数组。
  - labels: 饼图每个部分对应的标签。
  - autopct: 格式化字符串，用于显示每个部分的百分比，'%.0f%%'表示显示整数百分比。
  - radius: 饼图的半径大小。
  - textprops: 设置文本属性，这里设置字体大小为16。

plt.show()  # 显示图表。在交互式环境中，如Jupyter Notebook，这将在输出区域显示图表；在脚本中运行时，会打开一个窗口显示图表。
```

这段代码的作用是统计不同类别心跳声音数据集的文件数量，并将这些数量以饼图的形式可视化，从而直观地展示每个类别在总数据集中的比例。这对于快速了解数据集的分布情况非常有帮助。

### Visualizing random sample

In [ ]:
def visulize_random_sample(folder_name):
  #to hear the audio sample
  random_sample             = np.random.randint(0,len(os.listdir(folder_name)))
  sample_sound              = os.listdir(folder_name)[random_sample]
  sample_address            = folder_name + sample_sound
  sample_sound, sample_rate = librosa.load(sample_address)
  sample_audio              = ipd.Audio(sample_sound, rate=sample_rate)
  return sample_audio

中文注释：

```python
def visulize_random_sample(folder_name):  # 定义一个函数visulize_random_sample，它接受一个参数folder_name，即一个文件夹的路径。
  # to hear the audio sample  # 注释：以下代码用于播放随机选取的音频样本。
  random_sample = np.random.randint(0, len(os.listdir(folder_name)))  # 从0到folder_name目录下的文件数量（不包括目录）中随机选择一个整数，作为随机样本的索引。
  sample_sound = os.listdir(folder_name)[random_sample]  # 使用随机索引从folder_name目录中选取一个文件名。
  sample_address = folder_name + sample_sound  # 将选定的文件名与目录路径拼接，形成完整的文件路径。
  sample_sound, sample_rate = librosa.load(sample_address)  # 使用librosa.load()函数加载音频文件，返回音频信号数据sample_sound和对应的采样率sample_rate。
  sample_audio = ipd.Audio(sample_sound, rate=sample_rate)  # 使用IPython.display.Audio()函数创建一个音频对象，可以用于播放，sample_sound是音频信号数据，sample_rate是采样率。
  return sample_audio  # 函数返回创建的音频对象sample_audio，这样在调用该函数的地方就可以播放选定的随机音频样本。
```

这段代码定义了一个函数，用于随机选择并播放指定文件夹中的一个音频样本。这对于快速预览和听取数据集中的音频文件非常有用，尤其是在进行数据探索或质量检查时。

In [ ]:
visulize_random_sample(normal_data)

中文注释：

```python
visulize_random_sample(normal_data)  # 调用之前定义的函数visulize_random_sample，并传入参数normal_data。normal_data是一个包含正常心跳声音文件的目录路径。
# 函数将执行以下步骤：
# 1. 随机选择normal_data目录中的一个音频文件。
# 2. 使用librosa库加载这个音频文件，并获取音频信号和采样率。
# 3. 使用IPython.display库的Audio函数创建一个可以播放的音频对象。
# 4. 返回这个音频对象，调用者可以使用它来播放选中的音频样本。
```

当你执行这行代码时，它会触发函数`visulize_random_sample`的所有步骤，并最终播放normal_data目录下随机选择的一个正常心跳声音的音频样本。这个操作通常用在数据分析或机器学习项目中，用于验证数据集的质量和内容。

#### 1. Normal sound
Most normal heart rates at rest will be between about 60 and 100 beats (‘lub dub’s) per minute.

In [ ]:
# Choose random soud from normal folder
random_normal= np.random.randint(0,len(os.listdir(normal_data))) 
normal_sound = os.listdir(normal_data)[random_normal]
normal_sound_address = normal_data+normal_sound
normal_sound_sample,sample_rate = librosa.load(normal_sound_address)
ipd.Audio(normal_sound_sample,rate=sample_rate)

中文注释：

```python
# Choose random sound from normal folder  # 注释：从normal_data目录中随机选择一个声音文件。
random_normal = np.random.randint(0, len(os.listdir(normal_data)))  # 使用numpy的randint函数生成一个从0到normal_data目录下文件数量（不包括子目录）的随机整数，作为随机选择声音文件的索引。

normal_sound = os.listdir(normal_data)[random_normal]  # 根据生成的随机索引，从normal_data目录中选择一个声音文件的名称。

normal_sound_address = normal_data + normal_sound  # 将选择的声音文件名称与目录路径拼接，形成完整的文件路径。

normal_sound_sample, sample_rate = librosa.load(normal_sound_address)  # 使用librosa库的load函数加载声音文件，得到声音样本数据normal_sound_sample和对应的采样率sample_rate。

ipd.Audio(normal_sound_sample, rate=sample_rate)  # 使用IPython.display库的Audio函数创建一个音频对象，该对象可以用于播放声音。normal_sound_sample是音频样本数据，sample_rate是采样率。这行代码将触发音频播放。
```

这段代码的作用是从包含正常心跳声音的目录`normal_data`中随机选择一个音频文件，然后加载并播放这个音频样本。这对于快速检查数据集中的音频文件内容非常有用，尤其是在进行数据探索或演示时。

##### Waveform
Sound is the pressure of air propagates to our ear. Digital audio file is gotten from a sound sensor that can detects sound waves and converting it to electrical signals.

Specifically, it's telling us about the wave's displacement, and how it changes over time.

In [ ]:
plt.figure(figsize=(20,5))
librosa.display.waveplot(normal_sound_sample, sr = sample_rate)
plt.title("Normal Sound")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

中文注释：

```python
plt.figure(figsize=(20,5))  # 使用matplotlib的pyplot模块创建一个新的图形对象。figsize参数设置图形的大小，这里设置为宽20英寸，高5英寸。

librosa.display.waveplot(normal_sound_sample, sr=sample_rate)  # 使用librosa库的waveplot函数绘制音频波形图。normal_sound_sample是音频样本数据，sr=sample_rate指定了音频的采样率。

plt.title("Normal Sound")  # 设置图形的标题为"Normal Sound"。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("Amplitude")  # 设置y轴的标签为"Amplitude"，表示振幅轴。

plt.show()  # 显示图形。在交互式环境中，如Jupyter Notebook，这将在输出区域显示图形；在脚本中运行时，会打开一个窗口显示图形。
```

这段代码的作用是绘制并显示之前加载的`normal_sound_sample`音频样本的波形图。通过波形图，可以直观地观察音频信号随时间变化的情况，这对于音频分析和理解音频内容非常有帮助。

X axis, represents time. Y-axis measures displacement of air molecules.This is where amplitude comes in. It measures how much a molecule is displaced from its resting position.

##### Spectrum
A sound spectrum is a representation of a sound – usually a short sample of a sound – in terms of the amount of vibration at each individual frequency. It is usually presented as a graph of either power or pressure as a function of frequency. The power or pressure is usually measured in decibels and the frequency is measured in vibrations per second (or hertz, abbreviation Hz) or thousands of vibrations per second (kilohertz, abbreviation kHz).

The spectrum expresses the frequency composition of the sound and is obtained by analyzing the sound. A sound spectrum is usually represented in a coordinate plane where the frequency f is plotted along the axis of abscissas and the amplitude A, or intensity, of a harmonic component with a given frequency is plotted along the axis of ordinates.

In [ ]:
fft_normal = np.fft.fft(normal_sound_sample)
magnitude_normal = np.abs(fft_normal)
freq_normal = np.linspace(0,sample_rate, len(magnitude_normal)) 
half_freq = freq_normal[:int(len(freq_normal)/2)]
half_magnitude = magnitude_normal[:int(len(freq_normal)/2)]

plt.figure(figsize=(12,8))
plt.plot(half_freq,half_magnitude)
plt.title("Spectrum")
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.show()

中文注释：

```python
fft_normal = np.fft.fft(normal_sound_sample)  # 使用numpy的fft函数对音频样本normal_sound_sample进行快速傅里叶变换（FFT），得到频域信号。

magnitude_normal = np.abs(fft_normal)  # 计算FFT结果的绝对值，得到频域信号的幅度。

freq_normal = np.linspace(0, sample_rate, len(magnitude_normal))  # 使用numpy的linspace函数生成一个从0到sample_rate的等间隔采样点数组，采样点的数量与音频样本的帧数相同。

half_freq = freq_normal[:int(len(freq_normal)/2)]  # 由于FFT的结果是对称的，这里只取频率数组的一半，即从0到Nyquist频率的部分。

half_magnitude = magnitude_normal[:int(len(magnitude_normal)/2)]  # 同样，只取幅度数组的一半，与half_freq对应。

plt.figure(figsize=(12,8))  # 创建一个新的图形对象，并设置图形的大小为宽12英寸，高8英寸。

plt.plot(half_freq, half_magnitude)  # 绘制频率（half_freq）与幅度（half_magnitude）的关系图，即频谱图。

plt.title("Spectrum")  # 设置图形的标题为"Spectrum"。

plt.xlabel("Frequency")  # 设置x轴的标签为"Frequency"，表示频率轴。

plt.ylabel("Magnitude")  # 设置y轴的标签为"Magnitude"，表示幅度轴。

plt.show()  # 显示图形。这将触发图形的渲染和显示，如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是将音频样本从时域转换到频域，并绘制其频谱图。频谱图显示了音频信号在不同频率上的幅度，这有助于分析音频信号的频率成分，对于声音分析和处理非常重要。

##### Spectogram
For us, as human, we sense a sound not only on a particular time by its intensity, but also by its pitch. The pitch is the frequency of the sound - higher pitch corresponding to higher frequency and vice versa. So, to have a representation which is closer to our brain, we can add another dimension - the frequency - to our representation, which is the Spectrogram.

A spectrogram is a visual representation of the spectrum of frequencies of a signal as it varies with time. When applied to an audio signal, spectrograms are sometimes called sonographs, voiceprints, or voicegrams.

Spectrograms are used extensively in the fields of music, linguistics, sonar, radar, speech processing,seismology, and others. Spectrograms of audio can be used to identify spoken words phonetically, and to analyse the various calls of animals.it can be generated by an optical spectrometer, a bank of band-pass filters, by Fourier transform or by a wavelet transform.

In [ ]:
# STFT -> spectrogram
hop_length = 512 # in num. of samples
n_fft = 2048 # window in num. of samples

# calculate duration hop length and window in seconds
hop_length_duration = float(hop_length)/sample_rate
n_fft_duration = float(n_fft)/sample_rate

print("STFT hop length duration is: {}s".format(hop_length_duration))
print("STFT window duration is: {}s".format(n_fft_duration))

# perform stft
stft_normal = librosa.stft(normal_sound_sample, n_fft=n_fft, hop_length=hop_length)

# calculate abs values on complex numbers to get magnitude
spectrogram = np.abs(stft_normal)
log_spectrogram = librosa.amplitude_to_db(spectrogram)

# display spectrogram
plt.figure(figsize=(15,10))
librosa.display.specshow(log_spectrogram, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
#plt.set_cmap("YlOrBr")
plt.title("Spectrogram")

中文注释：

```python
# STFT -> spectrogram  # 注释：将使用短时傅里叶变换（STFT）来生成频谱图（spectrogram）。

hop_length = 512  # in num. of samples  # 设置hop_length，即STFT中每次移动的样本数。
n_fft = 2048  # window in num. of samples  # 设置n_fft，即STFT中使用的窗口大小（样本数）。

# calculate duration hop length and window in seconds  # 计算hop_length和n_fft在时间上的持续时长（秒）。
hop_length_duration = float(hop_length)/sample_rate  # 将hop_length转换为秒。
n_fft_duration = float(n_fft)/sample_rate  # 将n_fft转换为秒。

print("STFT hop length duration is: {}s".format(hop_length_duration))  # 打印hop_length的持续时间。
print("STFT window duration is: {}s".format(n_fft_duration))  # 打印n_fft的持续时间。

# perform stft  # 执行短时傅里叶变换。
stft_normal = librosa.stft(normal_sound_sample, n_fft=n_fft, hop_length=hop_length)  # 对音频样本normal_sound_sample进行STFT，得到其频谱。

# calculate abs values on complex numbers to get magnitude  # 计算复数的绝对值以获得幅度。
spectrogram = np.abs(stft_normal)  # 取STFT结果的绝对值，得到频谱图的幅度。

log_spectrogram = librosa.amplitude_to_db(spectrogram)  # 将幅度转换为分贝值，以获得对数频率功率谱。

# display spectrogram  # 显示频谱图。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(log_spectrogram, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示对数频谱图。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。
plt.ylabel("Frequency")  # 设置y轴的标签为"Frequency"，表示频率轴。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

#plt.set_cmap("YlOrBr")  # 注释掉的代码，用于设置颜色映射，这里使用的是YlOrBr（黄色至橙色至棕色）。

plt.title("Spectrogram")  # 设置图形的标题为"Spectrogram"。
```

这段代码的作用是通过短时傅里叶变换（STFT）生成音频样本的频谱图，并以图形的方式显示。频谱图是一种显示信号频率成分随时间变化的热图，广泛应用于声音分析和处理中。通过频谱图，可以更直观地观察音频信号在不同频率和时间上的分布情况。

we have an image that represents a sound. X-axis is for time, y-axis is for frequency and the color is for intensity

##### MFCCs
We can’t take the raw audio signal as input to our model because there will be a lot of noise in the audio signal. It is observed that extracting features from the audio signal and using it as input to the base model will produce much better performance than directly considering raw audio signal as input. MFCC is the widely used technique for extracting the features from the audio signal.

in sound processing, the mel-frequency cepstrum (MFC) is a representation of the short-term power spectrum of a sound, based on a linear cosine transform of a log power spectrum on a nonlinear mel scale of frequency.

Mel-frequency cepstral coefficients (MFCCs) are coefficients that collectively make up an MFC. They are derived from a type of cepstral representation of the audio clip (a nonlinear "spectrum-of-a-spectrum"). The difference between the cepstrum and the mel-frequency cepstrum is that in the MFC, the frequency bands are equally spaced on the mel scale, which approximates the human auditory system's response more closely than the linearly-spaced frequency bands used in the normal spectrum. This frequency warping can allow for better representation of sound, for example, in audio compression.

**MFCCs are commonly derived as follows:**

1- Take the Fourier transform of (a windowed excerpt of) a signal.

2- Map the powers of the spectrum obtained above onto the mel scale, using triangular overlapping windows or alternatively, cosine overlapping windows.

3- Take the logs of the powers at each of the mel frequencies.

4- Take the discrete cosine transform of the list of mel log powers, as if it were a signal.

5- The MFCCs are the amplitudes of the resulting spectrum.

In [ ]:
# MFCCs
# extract 25 MFCCs
MFCCs = librosa.feature.mfcc(normal_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)

# display MFCCs
plt.figure(figsize=(15,10))
librosa.display.specshow(MFCCs, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("MFCC coefficients")
plt.colorbar()
#plt.set_cmap("YlOrBr")
plt.title("MFCCs")

# show plots
plt.show()

中文注释：

```python
# MFCCs  # 注释：开始进行梅尔频率倒谱系数（Mel-Frequency Cepstral Coefficients，MFCCs）的提取。

# extract 25 MFCCs  # 注释：提取25个MFCCs特征。
MFCCs = librosa.feature.mfcc(normal_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)  # 使用librosa的mfcc函数从normal_sound_sample音频样本中提取25个MFCCs特征。sample_rate是采样率，n_fft和hop_length分别是STFT的窗口大小和步长。

# display MFCCs  # 注释：显示MFCCs特征。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(MFCCs, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示MFCCs特征矩阵。sr是采样率，hop_length是步长。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("MFCC coefficients")  # 设置y轴的标签为"MFCC coefficients"，表示MFCC系数。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

#plt.set_cmap("YlOrBr")  # 注释掉的代码，用于设置颜色映射，这里使用的是YlOrBr（黄色至橙色至棕色）。

plt.title("MFCCs")  # 设置图形的标题为"MFCCs"。

# show plots  # 注释：显示绘制的图形。
plt.show()  # 显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是从音频样本中提取MFCCs特征，并将这些特征以热图的形式可视化。MFCCs是一种在语音和音频信号处理中广泛使用的特征，它们能够捕捉到人类对声音感知的某些重要方面。通过MFCCs热图，可以观察到不同时间段内声音信号的频谱特性。

#### 2. Murmur sound
Heart murmurs sound as though there is a “whooshing, roaring, rumbling, or turbulent fluid” noise in one of two temporal locations: (1) between “lub” and “dub”, or (2) between “dub” and “lub”. They can be a symptom of many heart disorders, some serious.

In [ ]:
  # Choose random soud from murmur folder
random_murmur= np.random.randint(0,len(os.listdir(murmur_data))) 
murmur_sound = os.listdir(murmur_data)[random_murmur]
murmur_sound_address = murmur_data+murmur_sound
murmur_sound_sample,sample_rate = librosa.load(murmur_sound_address)
ipd.Audio(murmur_sound_sample,rate=sample_rate)

中文注释：

```python
# Choose random sound from murmur folder  # 注释：从murmur_data目录中随机选择一个声音文件。

random_murmur = np.random.randint(0, len(os.listdir(murmur_data)))  # 使用numpy的randint函数生成一个从0到murmur_data目录下文件数量（不包括目录）的随机整数，作为随机选择声音文件的索引。

murmur_sound = os.listdir(murmur_data)[random_murmur]  # 使用随机索引从murmur_data目录中选择一个声音文件的名称。

murmur_sound_address = murmur_data + murmur_sound  # 将选择的声音文件名称与目录路径拼接，形成完整的文件路径。

murmur_sound_sample, sample_rate = librosa.load(murmur_sound_address)  # 使用librosa库的load函数加载声音文件，得到声音样本数据murmur_sound_sample和对应的采样率sample_rate。

ipd.Audio(murmur_sound_sample, rate=sample_rate)  # 使用IPython.display库的Audio函数创建一个音频对象，该对象可以用于播放声音。murmur_sound_sample是音频样本数据，sample_rate是采样率。这行代码将触发音频播放。
```

这段代码的作用是从包含心脏杂音声音的目录`murmur_data`中随机选择一个音频文件，然后加载并播放这个音频样本。这对于快速检查和听取心脏杂音数据集中的音频文件内容非常有用，尤其是在进行数据探索或演示时。

##### Waveform


In [ ]:
plt.figure(figsize=(20,5))
librosa.display.waveplot(murmur_sound_sample, sr = sample_rate)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

中文注释：

```python
plt.figure(figsize=(20,5))  # 使用matplotlib的pyplot模块创建一个新的图形对象，设置图形的大小为宽20英寸和高5英寸。

librosa.display.waveplot(murmur_sound_sample, sr=sample_rate)  # 使用librosa库的waveplot函数绘制音频样本murmur_sound_sample的波形图。sr=sample_rate参数指定了音频的采样率。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("Amplitude")  # 设置y轴的标签为"Amplitude"，表示振幅轴。

plt.show()  # 调用pyplot模块的show函数来显示图形。在Jupyter Notebook中，这将在输出区域显示图形；在Python脚本中运行时，会弹出一个窗口显示图形。
```

这段代码的作用是绘制并显示之前加载的心脏杂音音频样本`murmur_sound_sample`的波形图。波形图可以直观地展示音频信号随时间变化的振幅，对于分析和理解音频内容非常有用，尤其是在进行医学信号分析，如心脏杂音的识别和分类时。

##### Spectrum

In [ ]:
fft_murmur = np.fft.fft(murmur_sound_sample)
magnitude_murmur = np.abs(fft_murmur)
freq_murmur = np.linspace(0,sample_rate, len(magnitude_murmur)) 
half_freq = freq_murmur[:int(len(freq_murmur)/2)]
half_magnitude = magnitude_murmur[:int(len(freq_murmur)/2)]

plt.figure(figsize=(12,8))
plt.plot(half_freq,half_magnitude)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.show()

中文注释：

```python
fft_murmur = np.fft.fft(murmur_sound_sample)  # 使用numpy的fft函数对心脏杂音音频样本murmur_sound_sample进行快速傅里叶变换（FFT），得到频域信号。

magnitude_murmur = np.abs(fft_murmur)  # 计算FFT结果的绝对值，得到频域信号的幅度。

freq_murmur = np.linspace(0, sample_rate, len(magnitude_murmur))  # 使用numpy的linspace函数生成一个从0到sample_rate的等间隔采样点数组，采样点的数量与音频样本的帧数相同，用于表示频率轴。

half_freq = freq_murmur[:int(len(freq_murmur)/2)]  # 由于FFT结果是对称的，这里只取频率数组的前一半，即从0到Nyquist频率的部分。

half_magnitude = magnitude_murmur[:int(len(magnitude_murmur)/2)]  # 同样，只取幅度数组的前一半，与half_freq对应，形成一半的频率幅度对。

plt.figure(figsize=(12,8))  # 创建一个新的图形对象，并设置图形的大小为宽12英寸，高8英寸。

plt.plot(half_freq, half_magnitude)  # 使用matplotlib的pyplot模块绘制频率（half_freq）和幅度（half_magnitude）之间的关系图，即一半的频谱图。

plt.xlabel("Frequency")  # 设置x轴的标签为"Frequency"，表示频率轴。

plt.ylabel("Magnitude")  # 设置y轴的标签为"Magnitude"，表示幅度轴。

plt.show()  # 调用pyplot模块的show函数来显示图形。这将触发图形的渲染和显示，如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是将心脏杂音音频样本从时域转换到频域，并绘制其频谱图。频谱图显示了音频信号在不同频率上的幅度，有助于分析音频信号的频率成分，对于声音分析和处理非常重要。

##### Spectogram

In [ ]:
# STFT -> spectrogram
hop_length = 512 # in num. of samples
n_fft = 2048 # window in num. of samples

# calculate duration hop length and window in seconds
hop_length_duration = float(hop_length)/sample_rate
n_fft_duration = float(n_fft)/sample_rate

print("STFT hop length duration is: {}s".format(hop_length_duration))
print("STFT window duration is: {}s".format(n_fft_duration))

# perform stft
stft_murmur = librosa.stft(murmur_sound_sample, n_fft=n_fft, hop_length=hop_length)

# calculate abs values on complex numbers to get magnitude
spectrogram_murmur = np.abs(stft_murmur)
log_spectrogram_murmur = librosa.amplitude_to_db(spectrogram_murmur)

# display spectrogram
plt.figure(figsize=(15,10))
librosa.display.specshow(log_spectrogram, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.set_cmap("plasma")
plt.title("Spectrogram_murmur")

中文注释：

```python
# STFT -> spectrogram  # 注释：将使用短时傅里叶变换（STFT）来生成频谱图（spectrogram）。

hop_length = 512  # in num. of samples  # 设置hop_length，即STFT中每次移动的样本数。
n_fft = 2048  # window in num. of samples  # 设置n_fft，即STFT中使用的窗口大小（样本数）。

# calculate duration hop length and window in seconds  # 计算hop_length和n_fft在时间上的持续时长（秒）。
hop_length_duration = float(hop_length)/sample_rate  # 将hop_length转换为秒。
n_fft_duration = float(n_fft)/sample_rate  # 将n_fft转换为秒。

print("STFT hop length duration is: {}s".format(hop_length_duration))  # 打印hop_length的持续时间。
print("STFT window duration is: {}s".format(n_fft_duration))  # 打印n_fft的持续时间。

# perform stft  # 执行短时傅里叶变换。
stft_murmur = librosa.stft(murmur_sound_sample, n_fft=n_fft, hop_length=hop_length)  # 对音频样本murmur_sound_sample进行STFT，得到其频谱。

# calculate abs values on complex numbers to get magnitude  # 计算复数的绝对值以获得幅度。
spectrogram_murmur = np.abs(stft_murmur)  # 取STFT结果的绝对值，得到频谱图的幅度。

log_spectrogram_murmur = librosa.amplitude_to_db(spectrogram_murmur)  # 将幅度转换为分贝值，以获得对数频率功率谱。

# display spectrogram  # 显示频谱图。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(log_spectrogram_murmur, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示对数频谱图。注意这里应该是log_spectrogram_murmur而不是log_spectrogram。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("Frequency")  # 设置y轴的标签为"Frequency"，表示频率轴。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("plasma")  # 设置颜色映射为"plasma"，这是一种从低到高亮度的颜色渐变。

plt.title("Spectrogram_murmur")  # 设置图形的标题为"Spectrogram_murmur"。

# 注意：在调用specshow函数时，应使用log_spectrogram_murmur而不是log_spectrogram，否则会显示之前定义的正常声音的频谱图。
```

这段代码的作用是通过短时傅里叶变换（STFT）生成心脏杂音音频样本的频谱图，并以图形的方式显示。频谱图是一种显示信号频率成分随时间变化的热图，广泛应用于声音分析和处理中。通过频谱图，可以更直观地观察音频信号在不同频率和时间上的分布情况。

##### MFCCs

In [ ]:
# MFCCs
# extract 25 MFCCs
MFCCs_murmur = librosa.feature.mfcc(murmur_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)

# display MFCCs
plt.figure(figsize=(15,10))
librosa.display.specshow(MFCCs, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("MFCC coefficients")
plt.colorbar()
plt.set_cmap("plasma")
plt.title("MFCCs")

# show plots
plt.show()

中文注释：

```python
# MFCCs  # 注释：开始进行梅尔频率倒谱系数（Mel-Frequency Cepstral Coefficients，MFCCs）的提取。

# extract 25 MFCCs  # 注释：提取25个MFCCs特征。
MFCCs_murmur = librosa.feature.mfcc(murmur_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)  # 使用librosa的mfcc函数从murmur_sound_sample音频样本中提取25个MFCCs特征。sample_rate是采样率，n_fft和hop_length分别是STFT的窗口大小和步长。

# display MFCCs  # 注释：显示MFCCs特征。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(MFCCs_murmur, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示MFCCs特征矩阵。注意这里应使用MFCCs_murmur而不是MFCCs，以显示当前正在处理的心脏杂音的MFCCs。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("MFCC coefficients")  # 设置y轴的标签为"MFCC coefficients"，表示MFCC系数。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("plasma")  # 设置颜色映射为"plasma"，这是一种颜色渐变方案，用于增强图形的视觉对比度。

plt.title("MFCCs_murmur")  # 设置图形的标题为"MFCCs_murmur"，更准确地反映正在显示的是心脏杂音的MFCCs。

# show plots  # 注释：显示绘制的图形。
plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

请注意，在显示MFCCs时，代码中有一个错误：`specshow(MFCCs, ...)` 应该是 `specshow(MFCCs_murmur, ...)`，以确保显示的是心脏杂音的MFCCs而不是先前计算的正常声音的MFCCs。同样，标题也应该是 "MFCCs_murmur" 以匹配显示的内容。

#### 3. Extrastole sound
• Extrasystole sounds may appear occasionally and can be identified because there is a heart sound that is out of rhythm involving extra or skipped heartbeats, e.g. a “lub-lub dub” or a “lub dub-dub”.

In [ ]:
# Choose random soud from extrastole folder
random_extrastole= np.random.randint(0,len(os.listdir(extrastole_data))) 
extrastole_sound = os.listdir(extrastole_data)[random_extrastole]
extrastole_sound_address = extrastole_data+extrastole_sound
extrastole_sound_sample,sample_rate = librosa.load(extrastole_sound_address)
ipd.Audio(extrastole_sound_sample,rate=sample_rate)

中文注释：

```python
# Choose random sound from extrastole folder  # 注释：从extrastole_data目录中随机选择一个声音文件。

random_extrastole = np.random.randint(0, len(os.listdir(extrastole_data)))  # 使用numpy的randint函数生成一个从0到extrastole_data目录下文件数量（不包括目录）的随机整数，作为随机选择声音文件的索引。

extrastole_sound = os.listdir(extrastole_data)[random_extrastole]  # 使用随机索引从extrastole_data目录中选择一个声音文件的名称。

extrastole_sound_address = extrastole_data + extrastole_sound  # 将选择的声音文件名称与目录路径拼接，形成完整的文件路径。

extrastole_sound_sample, sample_rate = librosa.load(extrastole_sound_address)  # 使用librosa库的load函数加载声音文件，得到声音样本数据extrastole_sound_sample和对应的采样率sample_rate。

ipd.Audio(extrastole_sound_sample, rate=sample_rate)  # 使用IPython.display库的Audio函数创建一个音频对象，该对象可以用于播放声音。extrastole_sound_sample是音频样本数据，sample_rate是采样率。这行代码将触发音频播放。
```

这段代码的作用是从包含额外心动过速（extrastole）声音的目录`extrastole_data`中随机选择一个音频文件，然后加载并播放这个音频样本。这对于快速检查和听取心动过速数据集中的音频文件内容非常有用，尤其是在进行数据探索或演示时。

##### Waveform

In [ ]:
plt.figure(figsize=(20,5))
librosa.display.waveplot(extrastole_sound_sample, sr = sample_rate)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

中文注释：

```python
plt.figure(figsize=(20,5))  # 使用matplotlib的pyplot模块创建一个新的图形对象，并设置图形的大小为宽20英寸和高5英寸。

librosa.display.waveplot(extrastole_sound_sample, sr=sample_rate)  # 使用librosa库的waveplot函数绘制音频样本extrastole_sound_sample的波形图。sr=sample_rate参数指定了音频的采样率。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("Amplitude")  # 设置y轴的标签为"Amplitude"，表示振幅轴。

plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是绘制并显示之前加载的心动过速音频样本`extrastole_sound_sample`的波形图。波形图可以直观地展示音频信号随时间变化的振幅，对于分析和理解音频内容非常有用，尤其是在进行医学信号分析，如心动过速的识别和分类时。

##### Spectrum

In [ ]:
fft_extrastole = np.fft.fft(extrastole_sound_sample)
magnitude_extrastole = np.abs(fft_extrastole)
freq_extrastole = np.linspace(0,sample_rate, len(magnitude_extrastole)) 
half_freq = freq_extrastole[:int(len(freq_extrastole)/2)]
half_magnitude = magnitude_extrastole[:int(len(freq_extrastole)/2)]

plt.figure(figsize=(12,8))
plt.plot(half_freq,half_magnitude)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.show()

中文注释：

```python
fft_extrastole = np.fft.fft(extrastole_sound_sample)  # 使用numpy的fft函数对心动过速音频样本extrastole_sound_sample进行快速傅里叶变换（FFT），得到频域信号。

magnitude_extrastole = np.abs(fft_extrastole)  # 计算FFT结果的绝对值，得到频域信号的幅度。

freq_extrastole = np.linspace(0, sample_rate, len(magnitude_extrastole))  # 使用numpy的linspace函数生成一个从0到sample_rate的等间隔采样点数组，采样点的数量与音频样本的帧数相同，用于表示频率轴。

half_freq = freq_extrastole[:int(len(freq_extrastole)/2)]  # 由于FFT结果是对称的，这里只取频率数组的前一半，即从0到Nyquist频率的部分。

half_magnitude = magnitude_extrastole[:int(len(magnitude_extrastole)/2)]  # 同样，只取幅度数组的前一半，与half_freq对应，形成一半的频率幅度对。

plt.figure(figsize=(12,8))  # 创建一个新的图形对象，并设置图形的大小为宽12英寸，高8英寸。

plt.plot(half_freq, half_magnitude)  # 使用matplotlib的pyplot模块绘制频率（half_freq）和幅度（half_magnitude）之间的关系图，即一半的频谱图。

plt.xlabel("Frequency")  # 设置x轴的标签为"Frequency"，表示频率轴。

plt.ylabel("Magnitude")  # 设置y轴的标签为"Magnitude"，表示幅度轴。

plt.show()  # 调用pyplot模块的show函数来显示图形。这将触发图形的渲染和显示，如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是将心动过速音频样本从时域转换到频域，并绘制其频谱图。频谱图显示了音频信号在不同频率上的幅度，有助于分析音频信号的频率成分，对于声音分析和处理非常重要。

##### Spectogram

In [ ]:
# STFT -> spectrogram
hop_length = 512 # in num. of samples
n_fft = 2048 # window in num. of samples

# calculate duration hop length and window in seconds
hop_length_duration = float(hop_length)/sample_rate
n_fft_duration = float(n_fft)/sample_rate

print("STFT hop length duration is: {}s".format(hop_length_duration))
print("STFT window duration is: {}s".format(n_fft_duration))

# perform stft
stft_extrastole = librosa.stft(extrastole_sound_sample, n_fft=n_fft, hop_length=hop_length)

# calculate abs values on complex numbers to get magnitude
spectrogram_extrastole = np.abs(stft_extrastole)
log_spectrogram_extrastole = librosa.amplitude_to_db(spectrogram_extrastole)

# display spectrogram
plt.figure(figsize=(15,10))
librosa.display.specshow(log_spectrogram, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.set_cmap("cividis")
plt.title("Spectrogram_extrastole")

中文注释：

```python
# STFT -> spectrogram  # 注释：将使用短时傅里叶变换（STFT）来生成频谱图（spectrogram）。

hop_length = 512  # in num. of samples  # 设置hop_length，即STFT中每次移动的样本数。
n_fft = 2048  # window in num. of samples  # 设置n_fft，即STFT中使用的窗口大小（样本数）。

# calculate duration hop length and window in seconds  # 计算hop_length和n_fft在时间上的持续时长（秒）。
hop_length_duration = float(hop_length)/sample_rate  # 将hop_length转换为秒。
n_fft_duration = float(n_fft)/sample_rate  # 将n_fft转换为秒。

print("STFT hop length duration is: {}s".format(hop_length_duration))  # 打印hop_length的持续时间。
print("STFT window duration is: {}s".format(n_fft_duration))  # 打印n_fft的持续时间。

# perform stft  # 执行短时傅里叶变换。
stft_extrastole = librosa.stft(extrastole_sound_sample, n_fft=n_fft, hop_length=hop_length)  # 对音频样本extrastole_sound_sample进行STFT，得到其频谱。

# calculate abs values on complex numbers to get magnitude  # 计算复数的绝对值以获得幅度。
spectrogram_extrastole = np.abs(stft_extrastole)  # 取STFT结果的绝对值，得到频谱图的幅度。

log_spectrogram_extrastole = librosa.amplitude_to_db(spectrogram_extrastole)  # 将幅度转换为分贝值，以获得对数频率功率谱。

# display spectrogram  # 显示频谱图。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(log_spectrogram_extrastole, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示对数频谱图。注意这里应该是log_spectrogram_extrastole而不是log_spectrogram。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("Frequency")  # 设置y轴的标签为"Frequency"，表示频率轴。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("cividis")  # 设置颜色映射为"cividis"，这是一种颜色渐变方案，用于增强图形的视觉对比度。

plt.title("Spectrogram_extrastole")  # 设置图形的标题为"Spectrogram_extrastole"。

# 注意：在调用specshow函数时，应使用log_spectrogram_extrastole而不是log_spectrogram，否则会显示之前定义的频谱图而不是当前心动过速的频谱图。
```

这段代码的作用是通过短时傅里叶变换（STFT）生成心动过速音频样本的频谱图，并以图形的方式显示。频谱图是一种显示信号频率成分随时间变化的热图，广泛应用于声音分析和处理中。通过频谱图，可以更直观地观察音频信号在不同频率和时间上的分布情况。

##### MFCCs

In [ ]:
# MFCCs
# extract 25 MFCCs
MFCCs_extrastole = librosa.feature.mfcc(extrastole_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)

# display MFCCs
plt.figure(figsize=(15,10))
librosa.display.specshow(MFCCs, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("MFCC coefficients")
plt.colorbar()
plt.set_cmap("cividis")
plt.title("MFCCs_extrastole")

# show plots
plt.show()


中文注释：

```python
# MFCCs  # 注释：开始进行梅尔频率倒谱系数（Mel-Frequency Cepstral Coefficients，MFCCs）的提取。

# extract 25 MFCCs  # 注释：提取25个MFCCs特征。
MFCCs_extrastole = librosa.feature.mfcc(extrastole_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)  # 使用librosa的mfcc函数从extrastole_sound_sample音频样本中提取25个MFCCs特征。sample_rate是采样率，n_fft和hop_length分别是STFT的窗口大小和步长。

# display MFCCs  # 注释：显示MFCCs特征。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(MFCCs_extrastole, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示MFCCs特征矩阵。注意这里应使用MFCCs_extrastole而不是MFCCs，以显示当前正在处理的心动过速的MFCCs。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("MFCC coefficients")  # 设置y轴的标签为"MFCC coefficients"，表示MFCC系数。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("cividis")  # 设置颜色映射为"cividis"，这是一种颜色渐变方案，用于增强图形的视觉对比度。

plt.title("MFCCs_extrastole")  # 设置图形的标题为"MFCCs_extrastole"，更准确地反映正在显示的是心动过速的MFCCs。

# show plots  # 注释：显示绘制的图形。
plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

请注意，在显示MFCCs时，代码中有一个错误：`specshow(MFCCs, ...)` 应该是 `specshow(MFCCs_extrastole, ...)`，以确保显示的是心动过速的MFCCs而不是先前计算的其他音频的MFCCs。同样，标题也应该是 "MFCCs_extrastole" 以匹配显示的内容。

#### 4. Artifact sound
• In the Artifact category there are a wide range of different sounds, including feedback squeals and echoes, speech, music and noise.

In [ ]:
# Choose random soud from artifact folder
random_artifact= np.random.randint(0,len(os.listdir(artifact_data))) 
artifact_sound = os.listdir(artifact_data)[random_artifact]
artifact_sound_address = artifact_data+artifact_sound
artifact_sound_sample,sample_rate = librosa.load(artifact_sound_address)
ipd.Audio(artifact_sound_sample,rate=sample_rate)

中文注释：

```python
# Choose random sound from artifact folder  # 注释：从artifact_data目录中随机选择一个声音文件。

random_artifact = np.random.randint(0, len(os.listdir(artifact_data)))  # 使用numpy的randint函数生成一个从0到artifact_data目录下文件数量（不包括目录）的随机整数，作为随机选择声音文件的索引。

artifact_sound = os.listdir(artifact_data)[random_artifact]  # 使用随机索引从artifact_data目录中选择一个声音文件的名称。

artifact_sound_address = artifact_data + artifact_sound  # 将选择的声音文件名称与目录路径拼接，形成完整的文件路径。

artifact_sound_sample, sample_rate = librosa.load(artifact_sound_address)  # 使用librosa库的load函数加载声音文件，得到声音样本数据artifact_sound_sample和对应的采样率sample_rate。

ipd.Audio(artifact_sound_sample, rate=sample_rate)  # 使用IPython.display库的Audio函数创建一个音频对象，该对象可以用于播放声音。artifact_sound_sample是音频样本数据，sample_rate是采样率。这行代码将触发音频播放。
```

这段代码的作用是从包含伪迹（artifact）声音的目录`artifact_data`中随机选择一个音频文件，然后加载并播放这个音频样本。这对于快速检查和听取伪迹数据集中的音频文件内容非常有用，尤其是在进行数据探索或演示时。

In [ ]:
artifact_sound_address

实际上，`artifact_sound_address` 是之前代码段中的一个变量，它代表了随机选择的伪迹声音文件的完整路径。这里并没有新的代码行，而是对之前定义的变量的一个引用。我将为这个变量提供中文注释：

```python
artifact_sound_address = artifact_data + artifact_sound  # 将伪迹声音所在的目录路径（artifact_data）与随机选择的声音文件名（artifact_sound）拼接，生成该声音文件的完整路径地址。
```

在之前的代码中，这个变量被用来加载声音文件，并可以用于后续的声音处理或分析。例如，使用 `librosa.load(artifact_sound_address)` 可以加载声音文件，然后可以计算该声音文件的短时傅里叶变换（STFT）、生成频谱图、MFCCs 等特征。如果需要再次使用这个变量来引用声音文件的路径，可以直接使用 `artifact_sound_address`。

##### Waveform

In [ ]:
plt.figure(figsize=(20,5))
librosa.display.waveplot(artifact_sound_sample, sr = sample_rate)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

中文注释：

```python
plt.figure(figsize=(20,5))  # 使用matplotlib的pyplot模块创建一个新的图形对象，并设置图形的大小为宽20英寸和高5英寸。

librosa.display.waveplot(artifact_sound_sample, sr=sample_rate)  # 使用librosa库的waveplot函数绘制音频样本artifact_sound_sample的波形图。sr=sample_rate参数指定了音频的采样率，确保波形图的时间轴正确映射。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴，用于表示音频信号随时间的变化。

plt.ylabel("Amplitude")  # 设置y轴的标签为"Amplitude"，表示振幅轴，用于表示不同时间点的音频信号强度。

plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是绘制并显示之前加载的伪迹（artifact）音频样本`artifact_sound_sample`的波形图。波形图可以直观地展示音频信号随时间变化的振幅，对于分析和理解音频内容，如伪迹的识别和分类，非常有用。

##### Spectrum

In [ ]:
fft_artifact = np.fft.fft(artifact_sound_sample)
magnitude_artifact = np.abs(fft_artifact)
freq_artifact = np.linspace(0,sample_rate, len(magnitude_artifact)) 
half_freq = freq_artifact[:int(len(freq_artifact)/2)]
half_magnitude = magnitude_artifact[:int(len(freq_artifact)/2)]

plt.figure(figsize=(12,8))
plt.plot(half_freq,half_magnitude)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.show()

中文注释：

```python
fft_artifact = np.fft.fft(artifact_sound_sample)  # 使用numpy的fft函数对伪迹音频样本artifact_sound_sample进行快速傅里叶变换（FFT），得到频域信号。

magnitude_artifact = np.abs(fft_artifact)  # 计算FFT结果的绝对值，得到频域信号的幅度。

freq_artifact = np.linspace(0, sample_rate, len(magnitude_artifact))  # 使用numpy的linspace函数生成一个从0到sample_rate的等间隔采样点数组，采样点的数量与音频样本的帧数相同，用于表示频率轴。

half_freq = freq_artifact[:int(len(freq_artifact)/2)]  # 由于FFT结果是对称的，这里只取频率数组的前一半，即从0到Nyquist频率的部分。

half_magnitude = magnitude_artifact[:int(len(magnitude_artifact)/2)]  # 同样，只取幅度数组的前一半，与half_freq对应，形成一半的频率幅度对。

plt.figure(figsize=(12,8))  # 创建一个新的图形对象，并设置图形的大小为宽12英寸，高8英寸。

plt.plot(half_freq, half_magnitude)  # 使用matplotlib的pyplot模块绘制频率（half_freq）和幅度（half_magnitude）之间的关系图，即一半的频谱图。

plt.xlabel("Frequency")  # 设置x轴的标签为"Frequency"，表示频率轴。

plt.ylabel("Magnitude")  # 设置y轴的标签为"Magnitude"，表示幅度轴。

plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是将伪迹音频样本从时域转换到频域，并绘制其频谱图。频谱图显示了音频信号在不同频率上的幅度，有助于分析音频信号的频率成分，对于声音分析和处理非常重要。

##### Spectogram

In [ ]:
# STFT -> spectrogram
hop_length = 512 # in num. of samples
n_fft = 2048 # window in num. of samples

# calculate duration hop length and window in seconds
hop_length_duration = float(hop_length)/sample_rate
n_fft_duration = float(n_fft)/sample_rate

print("STFT hop length duration is: {}s".format(hop_length_duration))
print("STFT window duration is: {}s".format(n_fft_duration))

# perform stft
stft_artifact = librosa.stft(artifact_sound_sample, n_fft=n_fft, hop_length=hop_length)

# calculate abs values on complex numbers to get magnitude
spectrogram_artifact = np.abs(stft_artifact)
log_spectrogram_artifact = librosa.amplitude_to_db(spectrogram_artifact)

# display spectrogram
plt.figure(figsize=(15,10))
librosa.display.specshow(log_spectrogram, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.set_cmap("magma")
plt.title("Spectrogram_artifacte")

以下是上述Python代码的中文注释，同时修正了一些错误：

```python
# STFT -> spectrogram  # 注释：将使用短时傅里叶变换（STFT）来生成频谱图（spectrogram）。

hop_length = 512  # in num. of samples  # 设置hop_length，即STFT中每次移动的样本数。
n_fft = 2048  # window in num. of samples  # 设置n_fft，即STFT中使用的窗口大小（样本数）。

# calculate duration hop length and window in seconds  # 计算hop_length和n_fft在时间上的持续时长（秒）。
hop_length_duration = float(hop_length)/sample_rate  # 将样本数转换为秒数，得到hop_length的持续时间。
n_fft_duration = float(n_fft)/sample_rate  # 将样本数转换为秒数，得到n_fft的持续时间。

print("STFT hop length duration is: {}s".format(hop_length_duration))  # 打印hop_length的持续时间。
print("STFT window duration is: {}s".format(n_fft_duration))  # 打印n_fft的持续时间。

# perform stft  # 执行短时傅里叶变换。
stft_artifact = librosa.stft(artifact_sound_sample, n_fft=n_fft, hop_length=hop_length)  # 对音频样本artifact_sound_sample进行STFT，得到其频谱。

# calculate abs values on complex numbers to get magnitude  # 计算复数的绝对值以获得幅度。
spectrogram_artifact = np.abs(stft_artifact)  # 取STFT结果的绝对值，得到频谱图的幅度。

log_spectrogram_artifact = librosa.amplitude_to_db(spectrogram_artifact)  # 将幅度转换为分贝值，以获得对数频率功率谱。

# display spectrogram  # 显示频谱图。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(log_spectrogram_artifact, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示对数频谱图。注意这里应使用log_spectrogram_artifact而不是log_spectrogram。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("Frequency")  # 设置y轴的标签为"Frequency"，表示频率轴。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("magma")  # 设置颜色映射为"magma"，这是一种颜色渐变方案，用于增强图形的视觉对比度。

plt.title("Spectrogram_artifact")  # 设置图形的标题为"Spectrogram_artifact"。注意标题中的"e"是多余的，应去除以匹配变量名。

# 注意：在调用specshow函数时，应使用log_spectrogram_artifact而不是log_spectrogram，否则会显示之前定义的频谱图而不是当前伪迹声音的频谱图。同时，标题中的"_artifacte"应更正为"_artifact"。
```

这段代码的作用是通过短时傅里叶变换（STFT）生成伪迹音频样本的频谱图，并以图形的方式显示。频谱图是一种显示信号频率成分随时间变化的热图，广泛应用于声音分析和处理中。通过频谱图，可以更直观地观察音频信号在不同频率和时间上的分布情况。

##### MFCCs

In [ ]:
# MFCCs
# extract 25 MFCCs
MFCCs_artifact = librosa.feature.mfcc(artifact_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)

# display MFCCs
plt.figure(figsize=(15,10))
librosa.display.specshow(MFCCs, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("MFCC coefficients")
plt.colorbar()
plt.set_cmap("magma")
plt.title("MFCCs_artifact")

# show plots
plt.show()

以下是上述Python代码的中文注释，并修正了一些错误：

```python
# MFCCs  # 注释：开始进行梅尔频率倒谱系数（Mel-Frequency Cepstral Coefficients，MFCCs）的提取。

# extract 25 MFCCs  # 注释：提取25个MFCCs特征。
MFCCs_artifact = librosa.feature.mfcc(artifact_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)  # 使用librosa的mfcc函数从artifact_sound_sample音频样本中提取25个MFCCs特征。sample_rate是采样率，n_fft和hop_length分别是STFT的窗口大小和步长。

# display MFCCs  # 注释：显示MFCCs特征。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(MFCCs_artifact, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示MFCCs特征矩阵。注意这里应使用MFCCs_artifact而不是MFCCs，以显示当前正在处理的伪迹声音的MFCCs。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("MFCC coefficients")  # 设置y轴的标签为"MFCC coefficients"，表示MFCC系数。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("magma")  # 设置颜色映射为"magma"，这是一种颜色渐变方案，用于增强图形的视觉对比度。

plt.title("MFCCs_artifact")  # 设置图形的标题为"MFCCs_artifact"，更准确地反映正在显示的是伪迹声音的MFCCs。

# show plots  # 注释：显示绘制的图形。
plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

请注意，在显示MFCCs时，代码中有一个错误：`specshow(MFCCs, ...)` 应该是 `specshow(MFCCs_artifact, ...)`，以确保显示的是伪迹声音的MFCCs而不是先前计算的其他音频的MFCCs。同样，标题也应该是 "MFCCs_artifact" 以匹配显示的内容。

#### 5. Extrahls sound
Extra heart sounds can be identified because there is an additional sound, e.g. a “lub-lub dub” or a “lub dub-dub”.

In [ ]:
# Choose random soud from extrahls folder
random_extrahls= np.random.randint(0,len(os.listdir(extrahls_data))) 
extrahls_sound = os.listdir(extrahls_data)[random_extrahls]
extrahls_sound_address = extrahls_data+extrahls_sound
extrahls_sound_sample,sample_rate = librosa.load(extrahls_sound_address)
ipd.Audio(extrahls_sound_sample,rate=sample_rate)

中文注释：

```python
# Choose random sound from extrahls folder  # 注释：从extrahls_data目录中随机选择一个声音文件。

random_extrahls = np.random.randint(0, len(os.listdir(extrahls_data)))  # 使用numpy的randint函数生成一个从0到extrahls_data目录下文件数量（不包括目录）的随机整数，作为随机选择声音文件的索引。

extrahls_sound = os.listdir(extrahls_data)[random_extrahls]  # 使用随机索引从extrahls_data目录中选择一个声音文件的名称。

extrahls_sound_address = extrahls_data + extrahls_sound  # 将选择的声音文件名称与目录路径拼接，形成完整的文件路径。

extrahls_sound_sample, sample_rate = librosa.load(extrahls_sound_address)  # 使用librosa库的load函数加载声音文件，得到声音样本数据extrahls_sound_sample和对应的采样率sample_rate。

ipd.Audio(extrahls_sound_sample, rate=sample_rate)  # 使用IPython.display库的Audio函数创建一个音频对象，该对象可以用于播放声音。extrahls_sound_sample是音频样本数据，sample_rate是采样率。这行代码将触发音频播放。
```

这段代码的作用是从包含额外高保真声音（extrahls）的目录`extrahls_data`中随机选择一个音频文件，然后加载并播放这个音频样本。这对于快速检查和听取额外高保真数据集中的音频文件内容非常有用，尤其是在进行数据探索或演示时。

##### Waveform

In [ ]:
plt.figure(figsize=(20,5))
librosa.display.waveplot(extrahls_sound_sample, sr = sample_rate)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

中文注释：

```python
plt.figure(figsize=(20,5))  # 使用matplotlib的pyplot模块创建一个新的图形对象，并设置图形的大小为宽20英寸和高5英寸，用于绘制波形图。

librosa.display.waveplot(extrahls_sound_sample, sr=sample_rate)  # 使用librosa库的waveplot函数绘制音频样本extrahls_sound_sample的波形图。sr=sample_rate参数指定了音频的采样率，确保波形图的时间轴按实际时间比例显示。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴，用于表示音频信号随时间的变化。

plt.ylabel("Amplitude")  # 设置y轴的标签为"Amplitude"，表示振幅轴，用于表示不同时间点的音频信号强度。

plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是绘制并显示之前加载的额外高保真（extrahls）音频样本`extrahls_sound_sample`的波形图。波形图可以直观地展示音频信号随时间变化的振幅，对于分析和理解音频内容非常有用，尤其是在进行声音分析和处理时。

##### Spectrum

In [ ]:
fft_extrahls = np.fft.fft(extrahls_sound_sample)
magnitude_extrahls = np.abs(fft_extrahls)
freq_extrahls = np.linspace(0,sample_rate, len(magnitude_extrahls)) 
half_freq = freq_extrahls[:int(len(freq_extrahls)/2)]
half_magnitude = magnitude_extrahls[:int(len(freq_extrahls)/2)]

plt.figure(figsize=(12,8))
plt.plot(half_freq,half_magnitude)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.show()

中文注释：

```python
fft_extrahls = np.fft.fft(extrahls_sound_sample)  # 使用numpy的fft函数对额外高保真音频样本extrahls_sound_sample进行快速傅里叶变换（FFT），得到频域信号。

magnitude_extrahls = np.abs(fft_extrahls)  # 计算FFT结果的绝对值，得到频域信号的幅度。

freq_extrahls = np.linspace(0, sample_rate, len(magnitude_extrahls))  # 使用numpy的linspace函数生成一个从0到sample_rate的等间隔采样点数组，采样点的数量与音频样本的帧数相同，用于表示频率轴。

half_freq = freq_extrahls[:int(len(freq_extrahls)/2)]  # 由于FFT结果是对称的，这里只取频率数组的前一半，即从0到Nyquist频率的部分。

half_magnitude = magnitude_extrahls[:int(len(magnitude_extrahls)/2)]  # 同样，只取幅度数组的前一半，与half_freq对应，形成一半的频率幅度对。

plt.figure(figsize=(12,8))  # 创建一个新的图形对象，并设置图形的大小为宽12英寸，高8英寸。

plt.plot(half_freq, half_magnitude)  # 使用matplotlib的pyplot模块绘制频率（half_freq）和幅度（half_magnitude）之间的关系图，即一半的频谱图。

plt.xlabel("Frequency")  # 设置x轴的标签为"Frequency"，表示频率轴。

plt.ylabel("Magnitude")  # 设置y轴的标签为"Magnitude"，表示幅度轴。

plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

这段代码的作用是将额外高保真音频样本从时域转换到频域，并绘制其频谱图。频谱图显示了音频信号在不同频率上的幅度，有助于分析音频信号的频率成分，对于声音分析和处理非常重要。

##### Spectogram

In [ ]:
# STFT -> spectrogram
hop_length = 512 # in num. of samples
n_fft = 2048 # window in num. of samples

# calculate duration hop length and window in seconds
hop_length_duration = float(hop_length)/sample_rate
n_fft_duration = float(n_fft)/sample_rate

print("STFT hop length duration is: {}s".format(hop_length_duration))
print("STFT window duration is: {}s".format(n_fft_duration))

# perform stft
stft_extrahls = librosa.stft(extrahls_sound_sample, n_fft=n_fft, hop_length=hop_length)

# calculate abs values on complex numbers to get magnitude
spectrogram_extrahls = np.abs(stft_extrahls)
log_spectrogram_extrahls = librosa.amplitude_to_db(spectrogram_extrahls)

# display spectrogram
plt.figure(figsize=(15,10))
librosa.display.specshow(log_spectrogram, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.set_cmap("inferno")
plt.title("Spectrogram_extrahlse")

以下是上述Python代码的中文注释，并包含了一些必要的修正：

```python
# STFT -> spectrogram  # 注释：将使用短时傅里叶变换（STFT）来生成频谱图（spectrogram）。

hop_length = 512  # in num. of samples  # 设置hop_length，即STFT中每次移动的样本数。
n_fft = 2048  # window in num. of samples  # 设置n_fft，即STFT中使用的窗口大小（样本数）。

# calculate duration hop length and window in seconds  # 计算hop_length和n_fft在时间上的持续时长（秒）。
hop_length_duration = float(hop_length)/sample_rate  # 将样本数转换为秒数，得到hop_length的持续时间。
n_fft_duration = float(n_fft)/sample_rate  # 将样本数转换为秒数，得到n_fft的持续时间。

print("STFT hop length duration is: {}s".format(hop_length_duration))  # 打印hop_length的持续时间。
print("STFT window duration is: {}s".format(n_fft_duration))  # 打印n_fft的持续时间。

# perform stft  # 执行短时傅里叶变换。
stft_extrahls = librosa.stft(extrahls_sound_sample, n_fft=n_fft, hop_length=hop_length)  # 对音频样本extrahls_sound_sample进行STFT，得到其频谱。

# calculate abs values on complex numbers to get magnitude  # 计算复数的绝对值以获得幅度。
spectrogram_extrahls = np.abs(stft_extrahls)  # 取STFT结果的绝对值，得到频谱图的幅度。

log_spectrogram_extrahls = librosa.amplitude_to_db(spectrogram_extrahls)  # 将幅度转换为分贝值，以获得对数频率功率谱。

# display spectrogram  # 显示频谱图。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(log_spectrogram_extrahls, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示对数频谱图。注意这里应使用log_spectrogram_extrahls而不是log_spectrogram。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("Frequency")  # 设置y轴的标签为"Frequency"，表示频率轴。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("inferno")  # 设置颜色映射为"inferno"，这是一种颜色渐变方案，用于增强图形的视觉对比度。

plt.title("Spectrogram_extrahls")  # 设置图形的标题为"Spectrogram_extrahls"。注意标题中的"e"是多余的，应去除以匹配变量名。

# 注意：在调用specshow函数时，应使用log_spectrogram_extrahls而不是log_spectrogram，否则会显示之前定义的频谱图而不是当前额外高保真声音的频谱图。同时，标题中的"_extrahlse"应更正为"_extrahls"。
```

这段代码的作用是通过短时傅里叶变换（STFT）生成额外高保真音频样本的频谱图，并以图形的方式显示。频谱图是一种显示信号频率成分随时间变化的热图，广泛应用于声音分析和处理中。通过频谱图，可以更直观地观察音频信号在不同频率和时间上的分布情况。

##### MFCCs

In [ ]:
# MFCCs
# extract 25 MFCCs
MFCCs_extrahls = librosa.feature.mfcc(extrahls_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)

# display MFCCs
plt.figure(figsize=(15,10))
librosa.display.specshow(MFCCs, sr=sample_rate, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("MFCC coefficients")
plt.colorbar()
plt.set_cmap("inferno")
plt.title("MFCCs_extrahls")

# show plots
plt.show()

以下是上述Python代码的中文注释，并包含了一些必要的修正：

```python
# MFCCs  # 注释：开始进行梅尔频率倒谱系数（Mel-Frequency Cepstral Coefficients，MFCCs）的提取。

# extract 25 MFCCs  # 注释：提取25个MFCCs特征。
MFCCs_extrahls = librosa.feature.mfcc(extrahls_sound_sample, sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=25)  # 使用librosa的mfcc函数从extrahls_sound_sample音频样本中提取25个MFCCs特征。sample_rate是采样率，n_fft和hop_length分别是STFT的窗口大小和步长。

# display MFCCs  # 注释：显示MFCCs特征。
plt.figure(figsize=(15,10))  # 创建一个新的图形对象，并设置图形的大小为宽15英寸，高10英寸。

librosa.display.specshow(MFCCs_extrahls, sr=sample_rate, hop_length=hop_length)  # 使用librosa的specshow函数显示MFCCs特征矩阵。注意这里应使用MFCCs_extrahls而不是MFCCs，以显示当前正在处理的额外高保真声音的MFCCs。

plt.xlabel("Time")  # 设置x轴的标签为"Time"，表示时间轴。

plt.ylabel("MFCC coefficients")  # 设置y轴的标签为"MFCC coefficients"，表示MFCC系数。

plt.colorbar()  # 显示颜色条，它提供了颜色映射到数值的参考。

plt.set_cmap("inferno")  # 设置颜色映射为"inferno"，这是一种颜色渐变方案，用于增强图形的视觉对比度。

plt.title("MFCCs_extrahls")  # 设置图形的标题为"MFCCs_extrahls"，更准确地反映正在显示的是额外高保真声音的MFCCs。

# show plots  # 注释：显示绘制的图形。
plt.show()  # 调用pyplot模块的show函数来显示图形。如果是在Jupyter Notebook中，图形会显示在输出中；如果是在Python脚本中运行，会弹出一个窗口显示图形。
```

请注意，在显示MFCCs时，代码中有一个错误：`specshow(MFCCs, ...)` 应该是 `specshow(MFCCs_extrahls, ...)`，以确保显示的是额外高保真声音的MFCCs而不是先前计算的其他音频的MFCCs。同样，标题也应该是 "MFCCs_extrahls" 以匹配显示的内容。

## Loading Data
Follwing function loop on every audio file and extract the mfcc features and the output is the a numpy array contain these mfcc's.

In [ ]:
def load_file_data (folder, file_names, duration=10, sr=22050):
    input_length=sr*duration
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            X, sr = librosa.load( sound_file, sr=sr, duration=duration) 
            dur = librosa.get_duration(y=X, sr=sr)
            # pad audio file same duration
            if (round(dur) < duration):
                print ("fixing audio lenght :", file_name)
                y = librosa.util.fix_length(X, input_length)                
            # extract normalized mfcc feature from data
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=25).T,axis=0)             
        except Exception as e:
            print("Error encountered while parsing file: ", file)        
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)
    return data

中文注释：

```python
def load_file_data(folder, file_names, duration=10, sr=22050):  # 定义一个函数load_file_data，用于加载音频文件数据并计算其MFCC特征。
    input_length = sr * duration  # 计算输入长度，即音频的期望样本数，基于采样率sr和持续时间duration。

    data = []  # 初始化一个空列表，用于存储所有的特征数据。

    for file_name in file_names:  # 遍历传入的文件名列表。
        try:
            sound_file = folder + file_name  # 拼接文件夹路径和文件名，形成完整的文件路径。
            print("load file ", sound_file)  # 打印正在加载的文件路径。

            X, sr = librosa.load(sound_file, sr=sr, duration=duration)  # 使用librosa.load()加载音频文件，sr参数指定采样率，duration参数限制加载音频的持续时间。

            dur = librosa.get_duration(y=X, sr=sr)  # 计算加载的音频样本的实际持续时间。

            # pad audio file same duration  # 如果音频文件的持续时间小于期望的duration，则进行填充。
            if (round(dur) < duration):
                print("fixing audio length :", file_name)  # 打印正在处理的文件名。
                y = librosa.util.fix_length(X, input_length)  # 将音频信号X的长度调整为input_length。

            # extract normalized mfcc feature from data  # 从音频数据中提取归一化的MFCC特征。
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=25).T, axis=0)  # 计算MFCC特征，取其转置的均值。

        except Exception as e:  # 如果在加载或处理音频文件时发生异常。
            print("Error encountered while parsing file: ", file)  # 打印错误信息和文件名。

        feature = np.array(mfccs).reshape([-1, 1])  # 将MFCC特征转换为numpy数组，并重塑为单个样本的格式。

        data.append(feature)  # 将计算出的MFCC特征添加到data列表中。

    return data  # 返回包含所有MFCC特征的列表。

# 注意：在except块中，打印错误信息时使用的变量是file，但应该使用file_name，这可能是一个错误。
```

这段代码定义了一个函数，用于从指定的文件夹加载多个音频文件，并为每个文件计算归一化的MFCC特征。如果音频文件的持续时间小于指定的duration，则会进行填充以保证所有音频样本的长度一致。计算出的MFCC特征将被存储在一个列表中，并在函数结束时返回。这对于音频分析和机器学习任务中的音频数据处理非常有用。

## Preprocessing :

### Encoding

In [ ]:
# simple encoding of categories, convert to only 3 types:
# Normal (Include extrahls and extrastole)
# Murmur
# Artifact
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Map label text to integer
CLASSES = ['artifact','murmur','normal']
NB_CLASSES=len(CLASSES)

# Map integer value to text labels
label_to_int = {k:v for v,k in enumerate(CLASSES)}
print (label_to_int)
print (" ")
int_to_label = {v:k for k,v in label_to_int.items()}
print(int_to_label)

中文注释：

```python
# simple encoding of categories, convert to only 3 types:  # 注释：简单编码类别，将其转换为仅3种类型。
# Normal (Include extrahls and extrastole)  # 正常（包括extrahls和extrastole）
# Murmur  # 杂音
# Artifact  # 伪迹

from sklearn.model_selection import train_test_split  # 导入train_test_split函数，用于分割数据集为训练集和测试集。
from sklearn import preprocessing  # 导入preprocessing模块，其中包含数据预处理的工具，如标签编码。

# Map label text to integer  # 将标签文本映射到整数值。
CLASSES = ['artifact', 'murmur', 'normal']  # 定义类别标签的列表，包含'artifact', 'murmur', 'normal'。

NB_CLASSES = len(CLASSES)  # 计算类别的数量，即CLASSES列表的长度。

# Map integer value to text labels  # 将整数值映射回文本标签。
label_to_int = {k: v for v, k in enumerate(CLASSES)}  # 创建一个字典，将类别标签映射到从0开始的整数。
print(label_to_int)  # 打印映射后的标签到整数的字典。
print(" ")  # 打印一个空行，为了输出的整洁。

int_to_label = {v: k for k, v in label_to_int.items()}  # 创建一个反向映射的字典，将整数映射回对应的类别标签。
print(int_to_label)  # 打印映射后的整数到标签的字典。
```

这段代码的作用是为机器学习任务准备标签编码。首先，它定义了三个类别标签，并创建了两个映射：一个将文本标签映射到整数，另一个将整数映射回文本标签。这样的映射通常用于将分类任务中的类别标签转换为模型可以处理的数值形式。

In [ ]:
# 22 KHz
SAMPLE_RATE = 22050
# seconds
MAX_SOUND_CLIP_DURATION=10

artifact_files = fnmatch.filter(os.listdir(artifact_data), 'artifact*.wav')
artifact_sounds = load_file_data (folder=artifact_data, file_names = artifact_files, duration=MAX_SOUND_CLIP_DURATION)
artifact_labels = [0 for items in artifact_files]

normal_files = fnmatch.filter(os.listdir(normal_data), 'normal*.wav')
normal_sounds = load_file_data(folder=normal_data,file_names=normal_files, duration=MAX_SOUND_CLIP_DURATION)
normal_labels = [2 for items in normal_sounds]

extrahls_files = fnmatch.filter(os.listdir(extrahls_data), 'extrahls*.wav')
extrahls_sounds = load_file_data(folder=extrahls_data,file_names=extrahls_files, duration=MAX_SOUND_CLIP_DURATION)
extrahls_labels = [2 for items in extrahls_sounds]

murmur_files = fnmatch.filter(os.listdir(murmur_data), 'murmur*.wav')
murmur_sounds = load_file_data(folder=murmur_data,file_names=murmur_files, duration=MAX_SOUND_CLIP_DURATION)
murmur_labels = [1 for items in murmur_files]


extrastole_files = fnmatch.filter(os.listdir(extrastole_data), 'extrastole*.wav')
extrastole_sounds = load_file_data(folder=extrastole_data,file_names=extrastole_files, duration=MAX_SOUND_CLIP_DURATION)
extrastole_labels = [2 for items in extrastole_files]

print ("Loading Done")

中文注释：

```python
# 22 KHz  # 注释：设置采样率为22,050 Hz，这是音频处理中常用的采样率之一。
SAMPLE_RATE = 22050
# seconds  # 注释：设置音频片段的最大持续时间为10秒。
MAX_SOUND_CLIP_DURATION = 10

# 使用fnmatch.filter函数找到artifact_data目录下所有以'artifact'开头并以'.wav'结尾的文件。
artifact_files = fnmatch.filter(os.listdir(artifact_data), 'artifact*.wav')
# 使用load_file_data函数加载artifact_files中的音频数据，限制每个音频的持续时间为MAX_SOUND_CLIP_DURATION。
artifact_sounds = load_file_data(folder=artifact_data, file_names=artifact_files, duration=MAX_SOUND_CLIP_DURATION)
# 为artifact_files中的每个文件创建一个标签列表，所有标签初始化为0，对应于'artifact'类别。
artifact_labels = [0 for items in artifact_files]

# 使用fnmatch.filter函数找到normal_data目录下所有以'normal'开头并以'.wav'结尾的文件。
normal_files = fnmatch.filter(os.listdir(normal_data), 'normal*.wav')
# 加载normal_files中的音频数据。
normal_sounds = load_file_data(folder=normal_data, file_names=normal_files, duration=MAX_SOUND_CLIP_DURATION)
# 为normal_files中的每个文件创建一个标签列表，所有标签初始化为2，对应于'normal'类别。
normal_labels = [2 for items in normal_files]

# 使用fnmatch.filter函数找到extrahls_data目录下所有以'extrahls'开头并以'.wav'结尾的文件。
extrahls_files = fnmatch.filter(os.listdir(extrahls_data), 'extrahls*.wav')
# 加载extrahls_files中的音频数据。
extrahls_sounds = load_file_data(folder=extrahls_data, file_names=extrahls_files, duration=MAX_SOUND_CLIP_DURATION)
# 为extrahls_files中的每个文件创建一个标签列表，所有标签初始化为2，因为'extrahls'被归类到'normal'类别。
extrahls_labels = [2 for items in extrahls_sounds]

# 使用fnmatch.filter函数找到murmur_data目录下所有以'murmur'开头并以'.wav'结尾的文件。
murmur_files = fnmatch.filter(os.listdir(murmur_data), 'murmur*.wav')
# 加载murmur_files中的音频数据。
murmur_sounds = load_file_data(folder=murmur_data, file_names=murmur_files, duration=MAX_SOUND_CLIP_DURATION)
# 为murmur_files中的每个文件创建一个标签列表，所有标签初始化为1，对应于'murmur'类别。
murmur_labels = [1 for items in murmur_files]

# 使用fnmatch.filter函数找到extrastole_data目录下所有以'extrastole'开头并以'.wav'结尾的文件。
extrastole_files = fnmatch.filter(os.listdir(extrastole_data), 'extrastole*.wav')
# 加载extrastole_files中的音频数据。
extrastole_sounds = load_file_data(folder=extrastole_data, file_names=extrastole_files, duration=MAX_SOUND_CLIP_DURATION)
# 为extrastole_files中的每个文件创建一个标签列表，所有标签初始化为2，因为'extrastole'被归类到'normal'类别。
extrastole_labels = [2 for items in extrastole_files]

print("Loading Done")  # 打印消息，表示音频数据加载完成。
```

这段代码的作用是从不同的音频数据子目录中加载音频文件，计算它们的MFCC特征，并为每个文件分配一个标签。每个类别的音频文件都被加载和处理，然后根据其类别分配一个整数值作为标签。标签0代表'artifact'，标签1代表'murmur'，标签2代表'normal'（包括'extrahls'和'extrastole'）。这些加载和标记的数据可以用于训练机器学习模型。

In [ ]:
# unlabel_datala files
Bunlabelledtest_files = fnmatch.filter(os.listdir(unlabel_data), 'Bunlabelledtest*.wav')
Bunlabelledtest_sounds = load_file_data(folder=unlabel_data,file_names=Bunlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
Bunlabelledtest_labels = [-1 for items in Bunlabelledtest_sounds]

Aunlabelledtest_files = fnmatch.filter(os.listdir(unlabel_data), 'Aunlabelledtest*.wav')
Aunlabelledtest_sounds = load_file_data(folder=unlabel_data,file_names=Aunlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
Aunlabelledtest_labels = [-1 for items in Aunlabelledtest_sounds]


print ("Loading of unlabel data done")

中文注释：

```python
# unlabel_data 文件  # 注释：处理未标记的测试数据。

# 使用fnmatch.filter函数找到unlabel_data目录下所有以'Bunlabelledtest'开头并以'.wav'结尾的文件。
Bunlabelledtest_files = fnmatch.filter(os.listdir(unlabel_data), 'Bunlabelledtest*.wav')
# 使用load_file_data函数加载Bunlabelledtest_files中的音频数据，限制每个音频的持续时间为MAX_SOUND_CLIP_DURATION。
Bunlabelledtest_sounds = load_file_data(folder=unlabel_data, file_names=Bunlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
# 为Bunlabelledtest_files中的每个文件创建一个标签列表，由于数据未标记，所有标签初始化为-1。
Bunlabelledtest_labels = [-1 for items in Bunlabelledtest_sounds]

# 使用fnmatch.filter函数找到unlabel_data目录下所有以'Aunlabelledtest'开头并以'.wav'结尾的文件。
Aunlabelledtest_files = fnmatch.filter(os.listdir(unlabel_data), 'Aunlabelledtest*.wav')
# 加载Aunlabelledtest_files中的音频数据。
Aunlabelledtest_sounds = load_file_data(folder=unlabel_data, file_names=Aunlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
# 为Aunlabelledtest_files中的每个文件创建一个标签列表，由于数据未标记，所有标签初始化为-1。
Aunlabelledtest_labels = [-1 for items in Aunlabelledtest_sounds]

print("Loading of unlabel data done")  # 打印消息，表示未标记数据的加载完成。
```

这段代码的作用是从未标记数据目录`unlabel_data`中加载两个不同类别的测试音频文件，计算它们的MFCC特征。由于这些文件是未标记的，所以它们的标签被设置为-1，表示它们在训练或评估模型时不会被用到，或者可以用于主动学习等场景，其中这些未标记的数据将被用于后续的标记过程。

### concatenation

In [ ]:
#combine set-a and set-b 
x_data = np.concatenate((artifact_sounds, normal_sounds,extrahls_sounds,murmur_sounds,extrastole_sounds))

y_data = np.concatenate((artifact_labels, normal_labels,extrahls_labels,murmur_labels,extrastole_labels))

test_x = np.concatenate((Aunlabelledtest_sounds,Bunlabelledtest_sounds))
test_y = np.concatenate((Aunlabelledtest_labels,Bunlabelledtest_labels))

print ("combined training data record: ",len(y_data), len(test_y))

中文注释：

```python
# combine set-a and set-b  # 注释：将集合A和集合B的数据进行合并。

# 使用numpy的concatenate函数将不同类别的声音特征数据合并到一起，形成训练数据集。
x_data = np.concatenate((artifact_sounds, normal_sounds, extrahls_sounds, murmur_sounds, extrastole_sounds))

# 使用numpy的concatenate函数将不同类别的标签数据合并到一起，形成训练标签集。
y_data = np.concatenate((artifact_labels, normal_labels, extrahls_labels, murmur_labels, extrastole_labels))

# 使用numpy的concatenate函数将未标记测试数据集A和B的声音特征数据合并到一起。
test_x = np.concatenate((Aunlabelledtest_sounds, Bunlabelledtest_sounds))

# 使用numpy的concatenate函数将未标记测试数据集A和B的标签数据合并到一起，所有标签都是-1。
test_y = np.concatenate((Aunlabelledtest_labels, Bunlabelledtest_labels))

# 打印训练数据记录和测试数据记录的数量，这里的len(y_data)是训练数据的标签数量，len(test_y)是测试数据的标签数量。
print("combined training data record: ", len(y_data), len(test_y))
```

这段代码的作用是将不同类别的音频特征数据和标签数据集合并成一个大的训练数据集和测试数据集。`x_data`变量包含了所有的声音特征，而`y_data`变量包含了对应的标签。对于测试数据集，`test_x`包含了所有未标记的声音特征，`test_y`包含了对应的未标记标签（-1）。最后，代码打印出训练数据集和测试数据集的记录数，这有助于确认数据集的规模。

In [ ]:
x_data.shape

上述代码实际上只有一行，它是对变量 `x_data` 使用 `.shape` 属性的调用。以下是这行代码的中文注释：

```python
x_data.shape  # 调用numpy数组的shape属性，用于获取数组的维度信息，即数组中每个轴的长度。在这个上下文中，x_data是包含所有音频特征的合并后的numpy数组，shape属性将返回一个元组，其中包含x_data数组的行数（样本数量）和每行的列数（每个样本的特征数量）。
```

这行代码通常用于了解数据集的大小，特别是在进行机器学习任务之前，确认数据是否已经正确地组织成了所需的形状。在这段代码的上下文中，`x_data` 应该包含了所有音频样本的MFCC特征，所以 `x_data.shape` 将告诉我们有多少个音频样本以及每个样本有多少个特征值。

### train_test_validation split

In [ ]:
# shuffle - whether or not to shuffle the data before splitting. If shuffle=False then stratify must be None.

# split data into Train, Validation and Test
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=42, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=42, shuffle=True)

# One-Hot encoding for classes
y_train = np.array(tf.keras.utils.to_categorical(y_train, len(CLASSES)))
y_test = np.array(tf.keras.utils.to_categorical(y_test, len(CLASSES)))
y_val = np.array(tf.keras.utils.to_categorical(y_val, len(CLASSES)))
test_y=np.array(tf.keras.utils.to_categorical(test_y, len(CLASSES)))

中文注释：

```python
# shuffle - whether or not to shuffle the data before splitting. If shuffle=False then stratify must be None.  # 注释：在分割数据之前是否打乱数据。如果shuffle设置为False，则stratify必须为None。

# split data into Train, Validation and Test  # 注释：将数据分割为训练集、验证集和测试集。
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=42, shuffle=True)  # 使用train_test_split函数将x_data和y_data分割为训练集和测试集，训练集占80%，设置随机状态为42以确保结果的可复现，并且数据会打乱。

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=42, shuffle=True)  # 进一步将训练集分割为训练集(x_train, y_train)和验证集(x_val, y_val)，同样训练集占80%，设置随机状态和打乱数据。

# One-Hot encoding for classes  # 注释：对类别进行One-Hot编码。
y_train = np.array(tf.keras.utils.to_categorical(y_train, len(CLASSES)))  # 对训练集标签进行One-Hot编码，使用CLASSES中类别的数量作为参数。
y_test = np.array(tf.keras.utils.to_categorical(y_test, len(CLASSES)))  # 对测试集标签进行One-Hot编码。
y_val = np.array(tf.keras.utils.to_categorical(y_val, len(CLASSES)))  # 对验证集标签进行One-Hot编码。

test_y = np.array(tf.keras.utils.to_categorical(test_y, len(CLASSES)))  # 对未标记测试数据集的标签进行One-Hot编码。
```

这段代码的作用是将原始的数据集分割为训练集、验证集和测试集，并确保在分割过程中数据会被打乱以提高模型的泛化能力。然后，它使用`tf.keras.utils.to_categorical`函数对标签进行One-Hot编码，这是多分类问题中常见的预处理步骤，它将整数编码的标签转换为二进制矩阵，使得每个类别都有一个唯一的二进制向量。这种编码方式对于使用softmax激活函数的多分类神经网络是必要的。最后，它将One-Hot编码后的标签转换为numpy数组。

### Correct imbalnced data using class weight

In [ ]:
TRAIN_IMG_COUNT = 578
COUNT_0 = 40  #artifact
COUNT_1 = 129 #murmur
COUNT_2 = 409 #normal
weight_for_0 = TRAIN_IMG_COUNT / (3 * COUNT_0)
weight_for_1 = TRAIN_IMG_COUNT / (3 * COUNT_1)
weight_for_2 = TRAIN_IMG_COUNT / (3 * COUNT_2)
class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}
class_weight

中文注释：

```python
TRAIN_IMG_COUNT = 578  # 定义TRAIN_IMG_COUNT，表示训练集中总的音频样本数量。

COUNT_0 = 40  # 定义COUNT_0，表示训练集中标签为0（即'artifact'类别）的样本数量。
COUNT_1 = 129  # 定义COUNT_1，表示训练集中标签为1（即'murmur'类别）的样本数量。
COUNT_2 = 409  # 定义COUNT_2，表示训练集中标签为2（即'normal'类别）的样本数量。

weight_for_0 = TRAIN_IMG_COUNT / (3 * COUNT_0)  # 计算类别0的权重，使用训练集中总样本数除以类别0样本数的三倍。
weight_for_1 = TRAIN_IMG_COUNT / (3 * COUNT_1)  # 计算类别1的权重，使用训练集中总样本数除以类别1样本数的三倍。
weight_for_2 = TRAIN_IMG_COUNT / (3 * COUNT_2)  # 计算类别2的权重，使用训练集中总样本数除以类别2样本数的三倍。

class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}  # 创建一个字典class_weight，将每个类别与其对应的权重关联起来。

class_weight  # 打印class_weight字典，展示每个类别的权重设置。
```

这段代码的作用是计算并设置不同类别的权重，以便在模型训练过程中对不平衡的数据集进行调整。权重是通过将训练集中的总样本数除以每个类别样本数的三倍来计算的，这样做的目的是尝试平衡不同类别之间的样本数量，使得模型不会偏向于多数类。最后，这些权重被存储在一个字典`class_weight`中，可以在训练机器学习模型时使用，以解决类别不平衡问题。

## LSTM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint,TensorBoard,ProgbarLogger
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

中文注释：

```python
from tensorflow.keras.models import Sequential  # 从tensorflow.keras.models模块导入Sequential模型，用于快速构建层的线性堆叠。

from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Flatten  # 从tensorflow.keras.layers模块导入多个常用的神经网络层：
  # Dense: 全连接层。
  # Dropout: 随机失活层，用于正则化。
  # Activation: 激活层，用于指定激活函数。
  # LSTM: 长短期记忆网络层，一种特殊的循环神经网络层。
  # Bidirectional: 双向层，常用于使LSTM或GRU层在正反两个方向上处理输入序列。
  # Flatten: 展平层，用于将多维输入一维化。

from tensorflow.keras.optimizers import Adam  # 从tensorflow.keras.optimizers模块导入Adam优化器，一种自适应学习率优化算法。

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, ProgbarLogger  # 从tensorflow.keras.callbacks模块导入多个回调函数：
  # EarlyStopping: 早期停止回调，用于在验证集性能不再提升时停止训练。
  # ReduceLROnPlateau: 在验证集性能停止提升时减少学习率。
  # ModelCheckpoint: 模型检查点回调，用于在训练过程中保存模型。
  # TensorBoard: 日志回调，用于将训练和验证数据写入日志文件，以便在TensorBoard中可视化。
  # ProgbarLogger: 进度条日志回调，显示训练进度。

from tensorflow.keras.regularizers import l2  # 从tensorflow.keras.regularizers模块导入L2正则化器，用于防止模型过拟合。

from sklearn import metrics  # 从scikit-learn库导入metrics模块，包含评估和比较模型的各种指标和函数。

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score  # 从scikit-learn的metrics模块导入多个评估分类模型性能的函数：
  # confusion_matrix: 混淆矩阵，显示真实类别与预测类别之间的关系。
  # classification_report: 分类报告，提供每个类别的精确度、召回率、F1分数等信息。
  # accuracy_score: 准确度分数，表示正确预测的样本数占总样本数的比例。
```

这段代码导入了构建和评估机器学习模型所需的各种工具和函数，包括模型架构、优化器、回调函数、正则化器以及性能评估指标。这些工具和函数为后续的模型构建、训练和评估提供了必要的支持。

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(128, dropout=0.05, recurrent_dropout=0.20, return_sequences=True), input_shape = (25,1)))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Flatten())

model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-4), metrics=['acc'])

model.summary()

中文注释：

```python
model = Sequential()  # 初始化Sequential模型，它是一个线性堆叠的层结构，用于快速构建模型。

# 添加一个双向的LSTM层，使用128个单元，设置dropout为0.05，recurrent_dropout（对递归状态的dropout）为0.20，
# return_sequences设置为True，表示返回LSTM每个时间步的输出，用于后续连接层的序列数据。
model.add(Bidirectional(LSTM(128, dropout=0.05, recurrent_dropout=0.20, return_sequences=True), input_shape=(25, 1)))

# 添加一个具有128个单元的全连接层，并使用ReLU激活函数。
model.add(Dense(128, activation='relu'))

# 添加一个Dropout层，设置Dropout率为0.3，用于正则化，减少过拟合。
model.add(Dropout(0.3))

# 再次添加两个具有128个单元的全连接层，并使用ReLU激活函数。
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

# 添加一个具有64个单元的全连接层，并使用ReLU激活函数。
model.add(Dense(64, activation='relu'))

# 添加一个Flatten层，将多维输入一维化，通常用于将二维的权重矩阵展平为一维，以便输入到全连接层。
model.add(Flatten())

# 添加最后一个全连接层，具有3个单元，并使用softmax激活函数，用于多分类问题。
model.add(Dense(3, activation='softmax'))

# 使用categorical_crossentropy作为损失函数，适合于多分类问题，使用Adam优化器，设置学习率为1e-4，并指定评估模型时使用的指标为准确度（'acc'）。
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-4), metrics=['acc'])

# 打印模型的详细摘要，包括每层的输出形状、参数数量等信息。
model.summary()
```

这段代码定义了一个基于LSTM和全连接层的神经网络模型，用于处理序列数据并进行多分类任务。模型首先使用一个双向LSTM层来学习序列数据中的特征，然后通过多个全连接层和Dropout层进一步提取特征并减少过拟合。最后，模型使用softmax激活函数输出每个类别的概率。模型摘要提供了模型结构的直观描述，有助于理解模型的复杂性和参数数量。

In [ ]:
weight_saver = ModelCheckpoint('set_a_weights.h5', monitor='val_loss', 
                               save_best_only=True, save_weights_only=True)
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.8**x)

中文注释：

```python
# 创建一个ModelCheckpoint回调，用于在训练过程中保存模型权重。
weight_saver = ModelCheckpoint('set_a_weights.h5',  # 指定保存模型权重的文件路径和文件名。
                               monitor='val_loss',  # 指定监控的目标为验证集的损失（'val_loss'）。
                               save_best_only=True,  # 仅在验证集损失最低时保存模型权重。
                               save_weights_only=True)  # 只保存模型的权重，而不保存整个模型结构。

# 创建一个LearningRateScheduler回调，用于在训练过程中动态调整学习率。
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.8**x)  # 使用Lambda函数定义学习率衰减策略，每次epoch后学习率会乘以0.8。
```

这段代码设置了两个回调（callbacks）：
1. `ModelCheckpoint`：它在训练过程中定期保存模型的权重，特别是当验证集损失最低时保存的权重。这有助于在训练结束后恢复训练过程，而无需从头开始。`save_best_only=True` 确保只有最佳模型权重被保存，而 `save_weights_only=True` 指示只保存模型权重而不保存完整的模型架构和优化器状态。
2. `LearningRateScheduler`：它允许在训练过程中根据epoch的数量调整学习率。这里使用了一个Lambda函数来定义学习率随epoch减少的策略，即每次epoch后学习率减少20%（乘以0.8）。这种学习率衰减是一种常见的技术，用于在训练初期使用较高的学习率快速收敛，在训练后期减小学习率以细化权重。

这些回调可以添加到模型的训练过程中，以提高训练效率和模型性能。

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

中文注释：

```python
callback = tf.keras.callbacks.EarlyStopping(  # 创建一个EarlyStopping回调对象，它是TensorFlow Keras中的一个回调函数。
    monitor='val_loss',  # 指定监控的目标为验证集的损失（'val_loss'），EarlyStopping将根据这个指标判断训练是否应该提前终止。
    patience=3  # 设置patience参数，它定义了在训练过程中，如果验证集损失（val_loss）在连续3个epoch中没有改善，则训练将被终止。
)
```

这段代码配置了一个`EarlyStopping`回调，它是一种在训练过程中用于避免过拟合和节省时间的常用技术。当模型在验证集上的性能在一定数量的epoch（这里是3个epoch）中没有提升时，`EarlyStopping`将终止训练过程。这个回调的主要参数是`monitor`，它指定了用于评估性能的指标，以及`patience`，它定义了在性能停止提升之前，允许模型训练的额外epoch数量。

In [ ]:
history=model.fit(x_train, y_train, 
                  batch_size=3, 
                  epochs=30,
                  class_weight=class_weight,
                  callbacks=[weight_saver, annealer],
                  validation_data=(x_val, y_val)) 

中文注释：

```python
history = model.fit(  # 使用model.fit方法开始训练模型。训练的结果和相关信息将存储在变量history中。
    x_train, y_train,  # 提供训练数据集的特征（x_train）和标签（y_train）。
    batch_size=3,  # 设置每个批次的样本数量为3。batch_size决定了每次更新模型权重时使用的样本数。
    epochs=30,  # 设置训练的总epoch数为30。每个epoch都会遍历一次完整的训练数据集。
    class_weight=class_weight,  # 使用之前定义的class_weight字典，为不同类别的样本分配不同的权重，以解决类别不平衡问题。
    callbacks=[weight_saver, annealer],  # 指定训练过程中将使用的回调函数列表，包括ModelCheckpoint和LearningRateScheduler。
    validation_data=(x_val, y_val)  # 提供验证数据集的特征（x_val）和标签（y_val），用于在每个epoch后评估模型性能并在必要时执行EarlyStopping。
)
```

这段代码的作用是启动模型的训练过程。它将使用指定的训练数据和参数，同时应用定义的回调函数。训练过程中，模型将在每个epoch后在验证集上进行评估，并且根据EarlyStopping的设置，如果验证损失在连续多个epoch没有改善，则训练可以提前终止。训练完成后，`history`对象将包含训练过程中的所有信息，如损失和准确度等，这些信息可以用于后续的模型分析和性能评估。

In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot()

中文注释：

```python
losses = pd.DataFrame(model.history.history)  # 使用pandas库创建一个DataFrame，其中包含模型训练过程中记录的历史信息。model.history.history是一个字典，包含了训练过程中的各种指标，如损失值（loss）和验证损失值（val_loss）。

losses[['loss', 'val_loss']].plot()  # 使用DataFrame对象的plot方法绘制训练损失（'loss'）和验证损失（'val_loss'）的图表。通过选择这两个键，图表将展示随着epoch增加，训练损失和验证损失的变化趋势。
```

这段代码的作用是将模型训练过程中的损失数据转换为一个便于分析的DataFrame格式，并绘制出损失值随着训练epoch变化的图表。这有助于我们直观地评估模型的学习进度和性能，以及判断是否存在过拟合或欠拟合的情况。图表中，通常训练损失会随着时间逐渐下降，而验证损失可能会在某个点后开始上升，这可能是过拟合的信号。

In [ ]:
losses[['acc','val_acc']].plot()

中文注释：

```python
losses[['acc', 'val_acc']].plot()  # 使用pandas DataFrame的plot方法绘制准确度（'accuracy'，简称'acc'）和验证集准确度（'val_acc'）的图表。通过选择这两个键，图表将展示随着训练epoch的增加，模型在训练集和验证集上的准确度变化趋势。
```

这段代码是在模型训练完成后执行的，用于可视化模型在训练过程中的准确度表现。`acc`代表训练集上的准确度，而`val_acc`代表验证集上的准确度。通过这个图表，我们可以观察模型的学习和泛化能力，即模型在训练集和验证集上的表现是否一致。如果训练准确度持续提高而验证准确度停滞不前或下降，这可能是一个过拟合的迹象。反之，如果两个准确度都较低，可能表明模型欠拟合，需要更多的训练或更复杂的模型结构。

## Prediction

In [ ]:
# make a prediction
y_pred = model.predict(x_test, batch_size=5)
#check scores
scores = model.evaluate(x_test, y_test)
print ("Model evaluation accuracy: ", round(scores[1]*100),"%")

中文注释：

```python
# make a prediction  # 注释：进行预测。
y_pred = model.predict(x_test, batch_size=5)  # 使用训练好的模型对测试集x_test进行预测。batch_size=5表示每次预测时使用5个样本进行计算，这有助于减小内存使用，尤其是在处理大型数据集或模型时。

# check scores  # 注释：检查模型在测试集上的性能评分。
scores = model.evaluate(x_test, y_test)  # 使用model.evaluate方法在测试集(x_test, y_test)上评估模型的性能，该方法返回一个包含损失值和准确度的列表。

print("Model evaluation accuracy: ", round(scores[1]*100), "%")  # 打印模型在测试集上的准确度评分。scores[1]是准确度索引，乘以100将其转换为百分比形式，并通过round函数四舍五入到最近的整数。
```

这段代码的作用是使用训练好的模型对测试集进行预测，并通过评估函数`model.evaluate`来获取模型在测试集上的损失和准确度。这有助于了解模型的最终性能，特别是在未见过的数据上的表现。打印出的准确度是模型性能的一个重要指标，它告诉我们模型在测试集上正确分类的样本所占的比例。

## Saving and loading model

In [ ]:
model.save('heart_sounds.h5')

中文注释：

```python
model.save('heart_sounds.h5')  # 使用model对象的save方法将整个模型保存到一个HDF5文件中。这里的文件名为'heart_sounds.h5'，通常这个文件会包含模型的结构、权重、优化器状态等所有信息，以便可以完全恢复模型的状态。
```

这段代码的作用是将训练好的模型保存到指定的文件中。HDF5是一种常用于存储和交换科学数据的文件格式，TensorFlow Keras也使用它来保存模型。保存后的模型文件可以在以后重新加载，用于进行预测、继续训练或分析模型结构，而无需重新训练。这在实际应用中非常有用，特别是当你需要部署模型或在不同环境中使用模型时。

In [ ]:
# prediction class 
y_pred = np.asarray(model.predict(x_test, batch_size=32))
y_pred = np.argmax(y_pred,axis=1)
print ("prediction test return :",y_pred[1], "-", int_to_label[y_pred[1]])

中文注释：

```python
# prediction class  # 注释：预测类别。

y_pred = np.asarray(model.predict(x_test, batch_size=32))  # 使用模型对测试集x_test进行预测。这里使用32个样本作为每个批次的batch_size进行预测，然后将预测结果转换为numpy数组。

y_pred = np.argmax(y_pred, axis=1)  # 使用numpy的argmax函数沿axis=1（列）找到每个样本预测概率最大的类别索引。这将one-hot编码的预测结果转换为实际的类别索引。

print("prediction test return :", y_pred[1], "-", int_to_label[y_pred[1]])  # 打印第二个样本的预测类别索引以及对应的文本标签。这里y_pred[1]是索引1的样本预测的类别索引，int_to_label[y_pred[1]]是该索引对应的文本标签。
```

这段代码的作用是将模型的预测概率转换为实际的类别索引，并打印出测试集中第二个样本的预测类别及其对应的文本标签。`np.argmax`函数在这里非常有用，因为它可以处理多分类问题中的one-hot编码输出，快速找到每个样本预测概率最大的类别。最后，通过`int_to_label`映射将类别索引转换为人类可读的文本标签。

In [ ]:
def preprocessing (file_path, duration=10, sr=22050):
  input_length=sr*duration
  process_file=[]
  X, sr = librosa.load(file_path, sr=sr, duration=duration) 
  dur = librosa.get_duration(y=X, sr=sr)
  # pad audio file same duration
  if (round(dur) < duration):
    y = librosa.util.fix_length(X, input_length)                
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40, n_fft=512,hop_length=2048).T,axis=0)
  feature = np.array(mfccs).reshape([-1,1])
  process_file.append(feature)
  process_file_array = np.asarray(process_file)
  return process_file_array

中文注释：

```python
def preprocessing(file_path, duration=10, sr=22050):  # 定义一个名为preprocessing的函数，用于预处理音频文件。接收参数file_path（文件路径）、duration（音频持续时间，默认为10秒）和sr（采样率，默认为22050Hz）。

  input_length = sr * duration  # 计算输入长度，即音频的期望样本数，基于采样率sr和持续时间duration。

  process_file = []  # 初始化一个空列表，用于存储处理后的音频特征。

  X, sr = librosa.load(file_path, sr=sr, duration=duration)  # 使用librosa.load()加载音频文件，限制加载音频的最大持续时间为duration。

  dur = librosa.get_duration(y=X, sr=sr)  # 计算加载的音频样本的实际持续时间。

  # pad audio file same duration  # 如果实际持续时间小于期望的duration，则对音频进行填充以匹配期望的持续时间。
  if (round(dur) < duration):
    y = librosa.util.fix_length(X, input_length)  # 使用librosa.util.fix_length()对音频信号进行填充或截断。

  # 计算MFCC特征，使用n_mfcc=40表示计算40个MFCC系数，n_fft=512和hop_length=2048分别设置FFT的大小和步长。
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40, n_fft=512, hop_length=2048).T, axis=0)

  feature = np.array(mfccs).reshape([-1, 1])  # 将MFCC特征转换为numpy数组，并重塑为单个样本的格式。

  process_file.append(feature)  # 将提取的MFCC特征添加到process_file列表中。

  process_file_array = np.asarray(process_file)  # 将process_file列表转换为numpy数组。

  return process_file_array  # 返回处理后的音频特征数组。
```

这段代码定义了一个函数，用于加载音频文件、计算其MFCC特征，并将特征重塑为适合机器学习模型输入的格式。它还处理了音频时长不足的情况，通过填充或截断音频以确保所有样本具有相同的长度。最后，函数返回一个包含音频特征的numpy数组，这个数组可以直接用于训练或评估机器学习模型。

In [ ]:
y_pred = np.asarray(model.predict(x_test, batch_size=32))
y_pred = np.argmax(y_pred,axis=1)

中文注释：

```python
y_pred = np.asarray(model.predict(x_test, batch_size=32))  # 使用模型的predict方法对测试集x_test进行预测。model.predict会返回预测结果，通常是模型输出层的原始得分或概率。这里使用32个样本作为每个批次的batch_size进行预测，然后将预测结果转换为numpy数组。

y_pred = np.argmax(y_pred, axis=1)  # 使用numpy的argmax函数沿axis=1（列）找到每个样本预测概率最大的类别索引。在多分类问题中，模型的输出是一个概率分布，每一列代表一个类别的概率。np.argmax(y_pred, axis=1)将这个概率分布转换为最可能的类别索引，即模型预测的类别。
```

这段代码的作用是根据模型的预测结果得到每个测试样本最可能的类别索引。在多分类问题中，模型的输出是一个概率矩阵，每一行代表一个样本，每一列代表对应类别的概率。`np.argmax`函数用于从这个概率矩阵中选择每个样本预测概率最高的类别索引，从而将概率形式的输出转换为具体的类别预测。这对于后续的评估和分类决策非常重要。

In [ ]:
target_names = ["artifact", "murmur","normal"]
print(classification_report(y_test.argmax(axis=1), y_pred, target_names = target_names))

中文注释：

```python
target_names = ["artifact", "murmur", "normal"]  # 定义一个列表，包含三个类别的名称："artifact", "murmur", "normal"。

print(classification_report(  # 使用sklearn.metrics中的classification_report函数打印一个分类报告，它将显示每个类别的精确度、召回率、F1分数以及其他指标。
    y_test.argmax(axis=1),  # 使用argmax函数从y_test的one-hot编码中提取真实类别的索引。y_test是测试集的标签，one-hot编码的，argmax(axis=1)将为每个样本找到最有可能的类别索引。
    y_pred,  # y_pred是模型预测的类别索引，由前面的代码计算得出。
    target_names=target_names  # 将target_names列表作为target_names参数传入，这样报告中将使用"artifact", "murmur", "normal"而不是默认的索引名。
))
```

这段代码的作用是评估模型在测试集上的性能，并以一种人类可读的格式打印出来。`classification_report`函数提供了一个非常详细的性能评估，包括每个类别的精确度、召回率和F1分数，以及宏平均和加权平均的分数。通过指定`target_names`参数，报告中的类别标签将显示为更易读的名称，而不是默认的索引值。这对于多分类问题尤其有用，因为它提供了对模型性能的全面了解。

## Treatment options for abnormal heartbeat sounds:

he treatment depends on the cause. You may need to make lifestyle changes, like increasing your activity level or changing your diet (for example, limiting your caffeine intake). If you smoke, your doctor will help you stop smoking. You might also require medication to control your abnormal heartbeat, as well as any secondary symptoms.

For serious abnormalities that don’t go away with behavioral changes or medication, your doctor can recommend:

1- cardiac catheterization to diagnose a heart problem

2- catheter ablation to destroy tissue that causes abnormal rhythms

3- cardioversion by medication or an electrical shock to the heart

4- implantation of a pacemaker or cardioverter defibrillator

5- surgery to correct an abnormality

Resources:

Breath sounds, https://medlineplus.gov/ency/article/007535.htm

Classifying Heart Sounds Challenge, http://www.peterjbentley.com/heartchallenge/#aboutdata

Audio Deep Learning Made Simple: Sound Classification, Step-by-Step, https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5

Heart Sounds Topic Review, https://www.healio.com/cardiology/learn-the-heart/cardiology-review/topic-reviews/heart-sounds

Deep Learning (for Audio) with Python course, https://youtube.com/playlist?list=PL-wATfeyAMNrtbkCNsLcpoAyBBRJZVlnf